In [3]:
# Load libraries
import networkx as nx
from networkx import *
import pandas as pd
import numpy as np
import math
import psycopg2
import sys
from pandas.io import sql
import sqlalchemy
import os
from sqlalchemy import create_engine, text
from sqlalchemy.types import INTEGER
from datetime import datetime
from configobj import ConfigObj
from tqdm.notebook import tqdm
tqdm.pandas()

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from functions.path_finder import solve_mw_path
from functions.utils import database

In [20]:
#Database connection information
config_path = "/home/jovyan/shared/rural_planner_refactoring/config_files/config_pe"

parser = ConfigObj(config_path)

sql_path = parser['sql_path']
country_folder = parser['country_folder']

owners = dict(parser['path_finder_fiber']['owners'])

schema = parser['path_finder_merge']['schema']

table_towers = parser['transport_by_tower_params']['table_infrastructure']
table_clusters = parser['clustering_params']['output_table']
output_table = parser['path_finder_mw']['output_table']
final_table = parser['path_finder_mw']['final_table']
table_line_of_sight = parser['path_finder_mw']['table_line_of_sight']

table_edges       = parser['path_finder_roads_params']['table_edges_roads']
table_nodes       = parser['path_finder_roads_params']['table_nodes_roads']
table_cluster_node_map = parser['path_finder_roads_params']['table_cluster_node_map']
table_clusters    = parser['clustering_params']['output_table']
table_planned_fiber_nodes =  parser['path_finder_fiber']['planned_fiber_nodes_table']
table_fiber_node_map = parser['path_finder_fiber']['fiber_node_map_table']
table_initial_quick_wins = parser['path_finder_fiber']['initial_qw_table']
output_table = parser['path_finder_fiber']['output_table']
final_table = parser['path_finder_fiber']['final_table']


table_towers      = parser['transport_by_tower_params']['table_infrastructure']
table_path_finder_fiber     = parser['path_finder_fiber']['final_table']
table_path_finder_radio     = parser['path_finder_mw']['final_table']
table_edges                 = parser['path_finder_roads_params']['table_edges_roads']
table_line_of_sight         = parser['path_finder_mw']['table_line_of_sight']
offset_cluster_road = 4
ultra_qw_radius = int(parser['path_finder_fiber']['ultra_qw_radius'])

table_output_1              = 'test_path_finder'
table_output_2              = 'test_path_finder_2'
table_output_microwave      = 'test_path_finder_mw'
table_output_microwave_2    = 'test_path_finder_mw_2'
table_output_basic          = 'output_path_finder_basic_test'
table_output_final          = 'output_path_finder_test'
#table_output_version        = table_output_final + '_v_' + str(d.year) + str(d.month).zfill(2) + str(d.day).zfill(2) + str(d.hour).zfill(2)
table_output_refined        = 'path_finder_cluster_aux'
table_output_clusters       = 'path_finder_clusters_1'

table_cluster_node_map      = parser['path_finder_roads_params']['table_cluster_node_map']
table_cluster_node_map_los  = parser['path_finder_roads_cl_los_params']['table_output']
table_clusters              = parser['clustering_params']['output_table']
#table_nodes                 = parser['path_finder_roads_params']['table_nodes_roads']
table_quick_wins_fiber      = 'path_finder_quick_wins_fiber'
table_quick_wins_radio      = 'path_finder_quick_wins_radio'
table_geom_fiber            = 'path_finder_fiber_geom'
table_output_crop           = 'test_path_finder_crop'
table_initial_quick_wins    = parser['path_finder_fiber']['initial_qw_table']
table_nodes = 'node_table_roads_aggregate'

In [34]:
##Load all needed inputs
query_path_towers_connect = sql_path + '/' + country_folder + '/' + 'path_finder_merge_load_towers_to_connect.sql'
query_path_id = sql_path + '/' + country_folder + '/' + 'path_finder_merge_load_max_id.sql'

with open(query_path_towers_connect) as file_towers, open(query_path_id) as file_id, database(parser) as db:
    query = file_towers.read()
    query_formatted = query.format(schema = schema,
                                   table_towers = table_towers,
                                   table_path_finder_fiber = table_path_finder_fiber,
                                   table_path_finder_radio = table_path_finder_radio,
                                   table_initial_quick_wins = table_initial_quick_wins)
    df_towers_to_connect = pd.read_sql_query(query_formatted, con = db)
    towers_to_connect = [str(x) for x in list(df_towers_to_connect['centroid'])]
   
    query = file_id.read()
    query_formatted = query.format(schema = schema,
                                   table_nodes_roads = table_nodes)
    node_id_max = pd.read_sql_query(query_formatted, con = db)['max'][0]

"""
query_path_towers = sql_path + '/' + country_folder + '/' + 'path_finder_merge_load_towers.sql'
query_path_ipt = sql_path + '/' + country_folder + '/' + 'path_finder_merge_load_ipt_towers.sql'
query_path_fiber = sql_path + '/' + country_folder + '/' + 'path_finder_merge_load_fiber.sql'
query_path_mw = sql_path + '/' + country_folder + '/' + 'path_finder_merge_load_mw.sql'
query_path_qw = sql_path + '/' + country_folder + '/' + 'path_finder_merge_load_ultra_qw.sql'
query_path_id = sql_path + '/' + country_folder + '/' + 'path_finder_merge_load_max_id.sql'

with open(query_path_towers) as file_towers, open(query_path_ipt) as file_ipt, open(query_path_fiber) as file_fiber, open(query_path_mw) as file_mw, open(query_path_qw) as file_qw, open(query_path_id) as file_id, database(parser) as db:
    query = file_towers.read()
    query_formatted = query.format(schema = schema,
                                   table_towers = table_towers)
    df_towers = pd.read_sql_query(query_formatted, con = db)
    
    query = file_ipt.read()
    query_formatted = query.format(schema = schema,
                                   table_towers = table_towers)
    df_towers_ipt = pd.read_sql_query(query_formatted, con = db)
    
    query = file_fiber.read()
    query_formatted = query.format(schema = schema,
                                   table_path_finder_fiber = table_path_finder_fiber,
                                   table_nodes_roads = table_nodes)
    df_towers_fiber = pd.read_sql_query(query_formatted, con = db)
    
    query = file_mw.read()
    query_formatted = query.format(schema = schema,
                                   table_path_finder_radio = table_path_finder_radio)
    df_towers_radio = pd.read_sql_query(query_formatted, con = db)
    
    query = file_qw.read()
    query_formatted = query.format(schema = schema,
                                   table_initial_quick_wins = table_initial_quick_wins)
    df_initial_quick_wins = pd.read_sql_query(query_formatted, con = db)
    initial_quick_wins = [str(x) for x in list(df_initial_quick_wins['centroid'])]
    
    query = file_id.read()
    query_formatted = query.format(schema = schema,
                                   table_nodes_roads = table_nodes)
    node_id_max = pd.read_sql_query(query_formatted, con = db)['max'][0]"""


"\nquery_path_towers = sql_path + '/' + country_folder + '/' + 'path_finder_merge_load_towers.sql'\nquery_path_ipt = sql_path + '/' + country_folder + '/' + 'path_finder_merge_load_ipt_towers.sql'\nquery_path_fiber = sql_path + '/' + country_folder + '/' + 'path_finder_merge_load_fiber.sql'\nquery_path_mw = sql_path + '/' + country_folder + '/' + 'path_finder_merge_load_mw.sql'\nquery_path_qw = sql_path + '/' + country_folder + '/' + 'path_finder_merge_load_ultra_qw.sql'\nquery_path_id = sql_path + '/' + country_folder + '/' + 'path_finder_merge_load_max_id.sql'\n\nwith open(query_path_towers) as file_towers, open(query_path_ipt) as file_ipt, open(query_path_fiber) as file_fiber, open(query_path_mw) as file_mw, open(query_path_qw) as file_qw, open(query_path_id) as file_id, database(parser) as db:\n    query = file_towers.read()\n    query_formatted = query.format(schema = schema,\n                                   table_towers = table_towers)\n    df_towers = pd.read_sql_query(query_

In [14]:
#Load edges
query_path = sql_path + '/' + country_folder + '/' + 'path_finder_fiber_load_edges.sql'

with open(query_path) as file, database(parser) as db:
    query = file.read()
    query_formatted = query.format(schema = schema, table_edges = table_edges, table_nodes = table_nodes)
    df_edges = pd.read_sql_query(query_formatted, con = db)

In [15]:
G = nx.Graph()

for z in range(0, len(df_edges)):
    G.add_edge(str(int(df_edges['node_1'].iloc[z])),
               str(int(df_edges['node_2'].iloc[z])),
               weight = df_edges['weight'].iloc[z])
    
graphs = list(nx.connected_component_subgraphs(G))

In [36]:
#Insert fiber nodes into the graph

#Load all fiber nodes and terminal nodes
query_path_fiber = sql_path + '/' + country_folder + '/' + 'path_finder_fiber_load_fiber.sql'

with open(query_path_fiber) as file_fiber, database(parser) as db:
    query = file_fiber.read()
    query_formatted = query.format(schema = schema,
                                   table_initial_quick_wins = table_initial_quick_wins,
                                   table_towers = table_towers,
                                   output_table = output_table,
                                   radius = ultra_qw_radius,
                                   table_edges = table_edges,
                                   table_nodes = table_nodes)
    df_edges_fiber = pd.read_sql_query(query_formatted, con = db)
    
nodes_graph = list(G.nodes())
df_edges_fiber_all = df_edges_fiber.ix[df_edges_fiber['node_2'].isin(nodes_graph),]

#We add the edges associated to fiber nodes to the graph
for z in range(0, len(df_edges_fiber_all)):
    G.add_edge(str(int(df_edges_fiber_all['node_1'].iloc[z])),
               str(int(df_edges_fiber_all['node_2'].iloc[z])),
               weight = df_edges_fiber_all['weight'].iloc[z])
    G.add_edge(str(int(df_edges_fiber_all['node_2'].iloc[z])),
               str(int(df_edges_fiber_all['node_1'].iloc[z])),
               weight = df_edges_fiber_all['weight'].iloc[z])

KeyboardInterrupt: 

In [38]:
#Load centroid to road LoS edges
query_path = sql_path + '/' + country_folder + '/' + 'path_finder_merge_load_los_centroids.sql'

with open(query_path) as file, database(parser) as db:
    query = file.read()
    query_formatted = query.format(schema = schema, 
                               table_cluster_node_map_los = table_cluster_node_map_los,
                               table_towers = table_towers)
    df_line_of_sight_centroid = pd.read_sql_query(query_formatted, con = db)
    

In [37]:
df_towers_to_connect_backup = df_towers_to_connect.copy(deep=True)

In [39]:
#We prepare data frames for prioritization
df_towers_to_connect = pd.merge(df_towers_to_connect, df_line_of_sight_centroid[['centroid', 'line_of_sight', 'error_flag']], how = 'left', on = 'centroid')
df_towers_to_connect_fiber = df_towers_to_connect.ix[((df_towers_to_connect['distance_to_road'] <= 1000) & (df_towers_to_connect['distance_to_road'].notnull()) 
                                                      | ((df_towers_to_connect['line_of_sight'] == True) & (df_towers_to_connect['error_flag'] == False)))]
df_towers_to_connect_fiber

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


,centroid,node_id,site,type,direct_population,distance_to_road,length_fiber_movistar,length_fiber_azteca,length_fiber_regional,length_fiber_third_party,...,fiber_node_regional,path_third_party,fiber_node_third_party,geom_microwave,geom_movistar,geom_azteca,geom_regional,geom_third_party,line_of_sight,error_flag
0,21779,22237.0,C_KIVINAKI,FEMTO,1236,807.364732,13.484013,13.666813,37.068374,255.362667,...,50697,"22237,22238,22240,22241,22242,22243,22245,2224...",50151,0102000020E6100000020000008718AF7955BE52C082AD...,0104000020E61000000A00000001010000005CFCB0B549...,0104000020E61000000A00000001010000005CFCB0B549...,0104000020E61000002200000001010000001D1243B267...,0104000020E61000006800000001010000005CFCB0B549...,True,False
2,18707,4287.0,C_SINAMI_SM21,FEMTO,563,429.932207,29.308406,29.875858,7.115735,393.641110,...,60316,"4287,4288,4289,4290,4291,4292,5765,4318,5689,5...",58726,0102000020E610000002000000B7627FD93D3353C094BC...,0104000020E6100000130000000101000000B9AE8A6B06...,0104000020E6100000150000000101000000B9AE8A6B06...,0104000020E6100000030000000101000000B9AE8A6B06...,0104000020E61000008B0000000101000000F69847B0E6...,True,False
5,23928,14606.0,C_NUEVO_TINGO,FEMTO,1748,255.195113,36.114243,41.351856,0.182528,159.091932,...,33144,"14606,14605,23351,23350,23348,23346,23345,2334...",58603,0102000020E610000002000000081EDFDE357A53C01D58...,0104000020E6100000130000000101000000811BB9C819...,0104000020E61000001200000001010000004A65020D26...,0101000020E6100000811BB9C8197A53C00CA89D4E167E...,0104000020E61000003E000000010100000008D64729A0...,False,False
11,20736,17.0,C_MANAS,FEMTO,519,190.648117,81.385854,80.646100,0.090915,449.029479,...,50997,"17,16,15,14,10848,10849,18,10851,10852,10853,1...",49291,0102000020E61000000200000014E81379924A53C0C6A7...,0104000020E61000001200000001010000000C03676D16...,0104000020E61000001200000001010000000C03676D16...,0101000020E61000009C94CC8EAD4A53C080D70C611631...,0104000020E6100000ED00000001010000009C94CC8EAD...,True,False
12,22961,6388.0,C_COCHANI_AY57,FEMTO,649,5766.626011,259.079013,71.897419,191.721401,4.985859,...,60073,"6388,6389,6390,49894",49894,0102000020E6100000020000005F7B6649805C52C006F5...,0104000020E61000006B00000001010000000CD34C03AC...,0104000020E6100000260000000101000000185DA5BBEB...,0104000020E61000004F000000010100000064F9646D41...,0104000020E6100000030000000101000000185DA5BBEB...,True,False
14,19283,1425.0,C_SAN_CRISTOBAL_MO29,FEMTO,1186,1876.199259,74.831253,125.797064,3.039241,328.892298,...,51253,"1425,1426,12724,12726,12727,714,715,716,4793,2...",49719,0102000020E61000000200000065AA605452AD51C04BC8...,0104000020E610000019000000010100000071F63DC135...,0104000020E610000035000000010100000000D89530B7...,0104000020E6100000020000000101000000421566FE37...,0104000020E61000006D0000000101000000AF8D2B6FC4...,True,False
17,21570,3697.0,C_TOTORAS_LA07,FEMTO,721,137.521119,86.039247,154.592310,206.013001,194.781355,...,62214,"3697,3698,3699,3700,3701,3703,3704,3696,3695,3...",58934,0102000020E6100000020000006E3480B740D853C04BC8...,0104000020E61000001F000000010100000072EFC2F5DF...,0104000020E6100000500000000101000000EC0AE995D7...,0104000020E610000049000000010100000035C3325C1B...,0104000020E610000048000000010100000035C3325C1B...,True,False
18,23885,19172.0,NR_EL_REPOSO_AM12,FEMTO,537,136.412625,16.202328,18.047588,12.540104,30.258670,...,33128,"19172,19173,19174,19175,19176,3373,3374,3375,3...",58446,0102000020E61000000200000068226C787AA753C02926...,0104000020E61000000700000001010000005C396ACE7F...,0104000020E61000000900000001010000005C396ACE7F...,0104000020E61000000500000001010000005C396ACE7F...,0104000020E61000000A0000000101000000673DDC0D8F...,True,False
22,21547,3740.0,C_PENACHI_LA27,FEMTO,647,292.959180,34.850546,104.489797,164.552226,153.320580,...,62214,"3740,3739,3738,1491,1490,1489,80,79,78,77,76,7...",58934,0102000020E6100000020000001FBAA0BE65DE53C0E71D...,0104000020E61000000F00000001010000008356CDD700...,0104000020E6100000400000000101000000EC0AE995D7...,0104000020E61000003F000000010

In [ ]:
def assign_optimal_fiber(row, offset):
    optimal_length = min(row['length_fiber_azteca'].astype(float) + offset_cluster_road))
    

df_towers_to_connect_fiber.join(df_towers_to_connect_fiber.apply(lambda(x,y,z): min(x.astype(float), (y.astype(float)+offset_cluster_road), axis=1, args=(owner, fiber_nodes_owner, G1, df_edges))

In [43]:
(df_towers_to_connect_fiber['length_fiber_movistar'].astype(float), (df_towers_to_connect_fiber['length_fiber_azteca'].astype(float) + offset_cluster_road), (df_towers_to_connect_fiber['length_fiber_third_party'].astype(float) + offset_cluster_road)).min()

AttributeError: 'tuple' object has no attribute 'min'

In [44]:
#We prepare data frames for prioritization
#df_towers_to_connect = pd.merge(df_towers_to_connect, df_line_of_sight_centroid[['centroid', 'line_of_sight', 'error_flag']], how = 'left', on = 'centroid')
df_towers_to_connect_fiber = df_towers_to_connect.ix[((df_towers_to_connect['distance_to_road'] <= 1000) & (df_towers_to_connect['distance_to_road'].notnull()) 
                                                      | ((df_towers_to_connect['line_of_sight'] == True) & (df_towers_to_connect['error_flag'] == False)))]

#2 km for Movistar is better than (up to) 2 + offset_cluster_road km for Azteca/Third parties

df_towers_to_connect_fiber['length_fiber_movistar_offset_cluster_road'] = df_towers_to_connect_fiber['length_fiber_movistar'].astype(float)
df_towers_to_connect_fiber['length_fiber_azteca_offset_cluster_road'] = df_towers_to_connect_fiber['length_fiber_azteca'].astype(float) + offset_cluster_road
df_towers_to_connect_fiber['length_fiber_third_party_offset_cluster_road'] = df_towers_to_connect_fiber['length_fiber_third_party'].astype(float) + offset_cluster_road
df_towers_to_connect_fiber['optimal_length'] = df_towers_to_connect_fiber[['length_fiber_movistar_offset_cluster_road', 'length_fiber_azteca_offset_cluster_road', 'length_fiber_third_party_offset_cluster_road']].min(axis = 1)

df_towers_to_connect_fiber['optimal_fiber_provider'] = 'MOVISTAR'
df_towers_to_connect_fiber.ix[abs(df_towers_to_connect_fiber['optimal_length'] - df_towers_to_connect_fiber['length_fiber_azteca_offset_cluster_road']) <= 0.00001, 'optimal_fiber_provider'] = 'AZTECA'
df_towers_to_connect_fiber.ix[abs(df_towers_to_connect_fiber['optimal_length'] - df_towers_to_connect_fiber['length_fiber_third_party_offset_cluster_road']) <= 0.00001, 'optimal_fiber_provider'] = 'THIRD PARTY'


df_towers_to_connect_fiber['optimal_fiber_node'] = df_towers_to_connect_fiber['fiber_node_movistar'] + node_id_max
df_towers_to_connect_fiber.ix[df_towers_to_connect_fiber['optimal_fiber_provider'] == 'AZTECA', 'optimal_fiber_node'] = df_towers_to_connect_fiber['fiber_node_azteca'] + node_id_max
df_towers_to_connect_fiber.ix[df_towers_to_connect_fiber['optimal_fiber_provider'] == 'THIRD PARTY', 'optimal_fiber_node'] = df_towers_to_connect_fiber['fiber_node_third_party'] + node_id_max


df_towers_to_connect_fiber = df_towers_to_connect_fiber.sort_values('optimal_length').reset_index(drop = True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/st

TypeError: must be str, not numpy.float64

In [ ]:
#Prioritization - first attempt just deploying the shortest path for each node

#Initialize variables
x = 0
df_edges_prioritization = pd.DataFrame(columns = ['deployment_id', 'site', 'fiber_node', 'site_road', 'fiber_node_road', 'node_1', 'node_2', 'weight', 'unlocked_sites', 'number_unlocked_sites', 'unlocked_nodes'])


for x in range(0, len(df_towers_to_connect_fiber)):
    
    if(x%1 == 0):
        print(x)
    
    #We reset index for each new iteration
    df_towers_to_connect_fiber = df_towers_to_connect_fiber.reset_index(drop = True)
    
    #If there is no fiber solution, we skip this iteration
    if(df_towers_to_connect_fiber['optimal_fiber_node'].isnull().iloc[x]):
        break

    #Current site and its fiber node (road nodes associated and original ids)
    site_iteration = str(int(df_towers_to_connect_fiber['node_id'].iloc[x]))
    fiber_node_iteration = str(int(df_towers_to_connect_fiber['optimal_fiber_node'].iloc[x]))

    tower_id_iteration = str(int(df_towers_to_connect_fiber['centroid'].iloc[x]))
    fiber_tower_iteration = str(int(fiber_node_iteration) - node_id_max)

    #Calculate shortest path for current node
    path = nx.dijkstra_path(G, site_iteration, fiber_node_iteration)
    length = nx.dijkstra_path_length(G, site_iteration, fiber_node_iteration)

    #Check if we unlocked any intermediate tower in the way we deployed the fiber (now it only checks towers in the set of towers to connect)
    towers_unlocked_iteration_fiber = df_towers_to_connect_fiber.ix[(df_towers_to_connect_fiber['node_id'].astype(int).astype(str).isin(path)) & 
                                                    (df_towers_to_connect_fiber['node_id'].astype(int).astype(str) != site_iteration)]['centroid'].values.tolist()


    #Check if connecting this site unlocks any other sites (belonging to the set of sites we want to connect) with only one extra MW hop (straight LoS)
    if(df_line_of_sight.ix[((df_line_of_sight['tower_id_1'] == int(tower_id_iteration)))].empty):
        towers_unlocked_1_iteration = []
    else:
        towers_unlocked_1_iteration = df_line_of_sight.ix[((df_line_of_sight['tower_id_1'] == int(tower_id_iteration)))]['tower_id_2'].values.tolist()

    if(df_line_of_sight.ix[((df_line_of_sight['tower_id_2'] == int(tower_id_iteration)))].empty):
        towers_unlocked_2_iteration = []
    else:
        towers_unlocked_2_iteration = df_line_of_sight.ix[((df_line_of_sight['tower_id_2'] == int(tower_id_iteration)))]['tower_id_1'].values.tolist()

    #Set of all unlocked sites
    list_towers = [str(x) for x in list(towers_unlocked_1_iteration + towers_unlocked_2_iteration + towers_unlocked_iteration_fiber)]
    #list_towers = [str(x) for x in list(towers_unlocked_1_iteration + towers_unlocked_2_iteration)]
    towers_unlocked_iteration = list_towers#list(towers_unlocked_1_iteration + towers_unlocked_2_iteration + towers_unlocked_iteration_fiber)

    #We remove to the sites to connect those that have already been unlocked
    #df_towers_to_connect_fiber = df_towers_to_connect_fiber.ix[(~df_towers_to_connect_fiber['node_id'].astype(int).astype(str).isin(path)) | (df_towers_to_connect_fiber['centroid'].astype(int).astype(str).isin(towers_unlocked_iteration))]

    #We create the data frame with all the edges needed to reach the current site
    path_iteration = []
    for y in range(0, len(path)-1):
        path_iteration = path_iteration + [[x, 
                                            tower_id_iteration, 
                                            fiber_tower_iteration, 
                                            site_iteration, 
                                            fiber_node_iteration, 
                                            path[y], 
                                            path[y+1], 
                                            G[path[y]][path[y+1]]['weight'], 
                                            towers_unlocked_iteration, 
                                            len(towers_unlocked_iteration) + len(path), 
                                            path]]

    df_edges_prioritization_iteration = pd.DataFrame.from_records(path_iteration)
    df_edges_prioritization_iteration.columns = ['deployment_id', 'site', 'fiber_node', 'site_road', 'fiber_node_road', 'node_1', 'node_2', 'weight', 'unlocked_sites', 'number_unlocked_sites', 'unlocked_nodes']

    df_edges_prioritization = df_edges_prioritization.append(df_edges_prioritization_iteration)



df_edges_prioritization

In [ ]:
#Export this first output

df_edges_prioritization.to_sql(table_output_1, con=engine, if_exists = 'replace', schema = schema, index = False, dtype={'unlocked_sites': sqlalchemy.types.VARCHAR})

query = """
UPDATE {schema}.{table_output_1} SET unlocked_sites = replace(unlocked_sites, '{{', '');
UPDATE {schema}.{table_output_1} SET unlocked_sites = replace(unlocked_sites, '}}', '');
UPDATE {schema}.{table_output_1} SET unlocked_nodes = replace(unlocked_nodes, '{{', '');
UPDATE {schema}.{table_output_1} SET unlocked_nodes = replace(unlocked_nodes, '}}', '');
"""

query_formatted = query.format(schema = schema,
                               schema_2 = schema_2,
                               table_output_1 = table_output_1).replace("[","").replace("]","")

connection = engine.connect()
engine.execute(query_formatted)

connection.close()

In [ ]:
#Second iteration: we prioritize by sites unlocked by km
#We load the data set with additional information

connection = engine.connect()


#This query loads for each site all road nodes where it unlocks any site (plus one additional row for itself and an empty one)
query = """
SELECT DISTINCT ON (site, unlocked_site)
P.*,
C1.cluster_weight AS direct_population,
CASE WHEN C2.cluster_weight IS NULL THEN 0
     ELSE C2.cluster_weight END AS unlocked_population
FROM (
    
    SELECT 
    A.site,
    A.fiber_node,
    A.site_road,
    A.fiber_node_road,
    A.weight,
    A.unlocked_sites,
    A.number_unlocked_sites,
    A.unlocked_nodes,
    M.centroid AS unlocked_site,
    A.unlocked_site AS unlocked_site_road
    FROM (
    SELECT 
    site, 
    fiber_node,
    site_road,
    fiber_node_road,
    SUM(weight) AS weight,
    unlocked_sites,
    number_unlocked_sites,
    unlocked_nodes,
    CASE WHEN unlocked_nodes = '' THEN ''
         ELSE TRIM(UNNEST(string_to_array(REPLACE(unlocked_nodes,'',''), ','))) END AS unlocked_site
    FROM {schema}.{table_output_1}
    GROUP BY 1,2,3,4,6,7,unlocked_nodes
    )A
    LEFT JOIN {schema}.{table_cluster_node_map_los} M
    ON A.unlocked_site = M.node_id::text
    
    
    UNION 
    
    SELECT 
    site, 
    fiber_node,
    site_road,
    fiber_node_road,
    SUM(weight) AS weight,
    unlocked_sites,
    number_unlocked_sites,
    unlocked_nodes,
    CASE WHEN unlocked_sites = '' THEN ''
         ELSE TRIM(UNNEST(string_to_array(REPLACE(unlocked_sites,'',''), ','))) END AS unlocked_site,
    '' AS unlocked_site_road
    FROM {schema}.{table_output_1}
    GROUP BY 1,2,3,4,6,7,unlocked_nodes
    
    ORDER BY site
) P
LEFT JOIN {schema_2}.{table_clusters} C1
ON P.site = C1.centroid
LEFT JOIN {schema_2}.{table_clusters} C2
ON P.unlocked_site = C2.centroid


UNION

SELECT 
site,
fiber_node,
site_road,
fiber_node_road,
weight,
unlocked_sites,
number_unlocked_sites,
unlocked_nodes,
'' AS unlocked_site,
'' AS unlocked_site_road,
direct_population,
0 AS unlocked_population
FROM 
(
        SELECT 
        P.*,
        C1.cluster_weight AS direct_population,
        CASE WHEN C2.cluster_weight IS NULL THEN 0
             ELSE C2.cluster_weight END AS unlocked_population
        FROM (
            
            SELECT 
            A.site,
            A.fiber_node,
            A.site_road,
            A.fiber_node_road,
            A.weight,
            A.unlocked_sites,
            A.number_unlocked_sites,
            A.unlocked_nodes,
            M.centroid AS unlocked_site
            FROM (
                SELECT 
                site, 
                fiber_node,
                site_road,
                fiber_node_road,
                SUM(weight) AS weight,
                unlocked_sites,
                number_unlocked_sites,
                unlocked_nodes,
                CASE WHEN unlocked_nodes = '' THEN ''
                     ELSE TRIM(UNNEST(string_to_array(REPLACE(unlocked_nodes,'',''), ','))) END AS unlocked_site
                FROM {schema}.{table_output_1} 
                GROUP BY 1,2,3,4,6,7,unlocked_nodes
            )A
            LEFT JOIN {schema}.{table_cluster_node_map} M
            ON A.unlocked_site = M.node_id::text
            
            
            UNION 
            
            SELECT 
            site, 
            fiber_node,
            site_road,
            fiber_node_road,
            SUM(weight) AS weight,
            unlocked_sites,
            number_unlocked_sites,
            unlocked_nodes,
            CASE WHEN unlocked_sites = '' THEN ''
                 ELSE TRIM(UNNEST(string_to_array(REPLACE(unlocked_sites,'',''), ','))) END AS unlocked_site
            FROM {schema}.{table_output_1}
            GROUP BY 1,2,3,4,6,7,unlocked_nodes
            
            ORDER BY site
        ) P
        LEFT JOIN {schema_2}.{table_clusters} C1
        ON P.site = C1.centroid
        LEFT JOIN {schema_2}.{table_clusters} C2
        ON P.unlocked_site = C2.centroid
) A
WHERE number_unlocked_sites > 0

ORDER BY site, unlocked_site, unlocked_site_road

"""
    
query_formatted = query.format(schema = schema,
                               schema_2 = schema_2,
                               table_nodes_roads = table_nodes_roads,
                               table_clusters = table_clusters,
                               table_cluster_node_map = table_cluster_node_map,
                               table_cluster_node_map_los = table_cluster_node_map_los,
                               table_output_1 = table_output_1).replace("[","").replace("]","")
connection = engine.connect()


df_prioritization_1 = pd.read_sql_query(query_formatted, con = engine)
df_prioritization_1['fiber_node'] = df_prioritization_1['fiber_node'].astype('float').astype('int')

connection.close()

df_prioritization_1 = df_prioritization_1.ix[(~ df_prioritization_1['unlocked_site'].isin(towers_quick_wins_radio)) & 
                       (~ df_prioritization_1['unlocked_site'].isin(towers_quick_wins_fiber)) & 
                       (~ df_prioritization_1['unlocked_site'].isin(initial_quick_wins))]

df_prioritization_1
#print(query_formatted)

In [ ]:
#STILL IMPROVABLE!!!!!!! DYNAMIC PRIORITIZATION

#Initialize variables
prioritization_list = []
unlocked_sites = []
prioritization_and_density = []
x = -1

df_towers_to_connect_fiber = df_towers_to_connect_fiber_backup.copy(deep = True)
G = G_backup.copy()
df_edges_prioritization = pd.DataFrame(columns = ['deployment_id', 'site', 'fiber_node', 'site_road', 'fiber_node_road', 'node_1', 'node_2', 'weight', 'old_weight','unlocked_sites', 'number_unlocked_sites', 'unlocked_nodes'])

#We start deploying all sites until there are no more sites to deploy
while(not(df_prioritization_1.empty)):
    x = x + 1
    if(x%10 == 0):
        print(x)
    
    #Data frame for current iteration
    df_prioritization_1_iteration = df_prioritization_1.copy(deep = True)
    df_prioritization_1_iteration = df_prioritization_1_iteration.fillna(0)

    #We calculate the population unlocked for each deployment (direct and indirect) as well as the population density for each site
    df_prioritization_1_iteration = df_prioritization_1_iteration.groupby('site').sum().reset_index()
    df_prioritization_1_iteration = pd.merge(df_prioritization_1, df_prioritization_1_iteration[['site', 'unlocked_population']], how = 'right', on = 'site')
    df_prioritization_1_iteration = df_prioritization_1_iteration.drop('unlocked_population_x', axis = 1)
    df_prioritization_1_iteration = df_prioritization_1_iteration.rename(index=str, columns={"unlocked_population_y": "unlocked_population"})
    
    df_prioritization_1_iteration['population'] = df_prioritization_1_iteration['direct_population']+df_prioritization_1_iteration['unlocked_population']
    df_prioritization_1_iteration['population'] = df_prioritization_1_iteration['unlocked_population']
    df_prioritization_1_iteration['population_density'] = df_prioritization_1_iteration['population']/df_prioritization_1_iteration['weight']
    
    #We order by population density to select the most dense deployment
    df_prioritization_1_iteration = df_prioritization_1_iteration.drop_duplicates('site').sort_values('population_density', ascending = False).reset_index(drop = True)
    
    
    #We select the most dense deployment and deploy it 
    #(it is not actually the most dense. We still do not account for previous deployments)
    #Explanation: when we start deploying, we update the weight of the paths deployed. This may cause that future deployments 
    #find a different path that is more optimal.
    #When this happens, we are not actually deploying the most dense sites, but the mose dense sites based on the previous 
    #solution. 
    #To solve this, we would have to re-calculate the solution for all remaining sites and then prioritize by density
    
    #Current site to deploy, fiber node to use as source and density of the deployment (in two references: graph-wise and 
    # Rural Planner-wise)
    tower_id_iteration = df_prioritization_1_iteration['site'].iloc[0]
    fiber_node_iteration = str(int(df_prioritization_1_iteration['fiber_node_road'].iloc[0]))
    
    site_iteration = str(int(df_prioritization_1_iteration['site_road'].iloc[0]))
    fiber_tower_iteration = str(int(df_prioritization_1_iteration['fiber_node'].iloc[0]))

    #Calculate shortest path for current node and its length
    path = nx.dijkstra_path(G, site_iteration, fiber_node_iteration)
    length = nx.dijkstra_path_length(G, site_iteration, fiber_node_iteration)
    length_old = nx.dijkstra_path_length(G_backup, site_iteration, fiber_node_iteration)

    #Check if we unlocked any intermediate tower in the way we deployed the fiber (now it only checks towers in the set of towers to connect)
    towers_unlocked_iteration_fiber = df_towers_to_connect_fiber.ix[(df_towers_to_connect_fiber['node_id'].astype(int).astype(str).isin(path)) & 
                                                    (df_towers_to_connect_fiber['node_id'].astype(int).astype(str) != site_iteration)]['centroid'].values.tolist()


    #Check if connecting this site unlocks any other sites (belonging to the set of sites we want to connect)
    if(df_line_of_sight.ix[((df_line_of_sight['tower_id_1'] == int(tower_id_iteration)))].empty):
        towers_unlocked_1_iteration = []
    else:
        towers_unlocked_1_iteration = df_line_of_sight.ix[((df_line_of_sight['tower_id_1'] == int(tower_id_iteration)))]['tower_id_2'].values.tolist()

    if(df_line_of_sight.ix[((df_line_of_sight['tower_id_2'] == int(tower_id_iteration)))].empty):
        towers_unlocked_2_iteration = []
    else:
        towers_unlocked_2_iteration = df_line_of_sight.ix[((df_line_of_sight['tower_id_2'] == int(tower_id_iteration)))]['tower_id_1'].values.tolist()

    #Set of all unlocked sites
    list_towers = [str(x) for x in list(towers_unlocked_1_iteration + towers_unlocked_2_iteration + towers_unlocked_iteration_fiber)]
    #list_towers = [str(x) for x in list(towers_unlocked_1_iteration + towers_unlocked_2_iteration)]
    towers_unlocked_iteration = list_towers
    
    #We create the data frame with all the edges needed to reach the current site and update the weight of 
    #the edges used to zero
    path_iteration = []
    for y in range(0, len(path)-1):
        path_iteration = path_iteration + [[x, 
                                            tower_id_iteration, 
                                            fiber_tower_iteration, 
                                            site_iteration, 
                                            fiber_node_iteration, 
                                            path[y], 
                                            path[y+1],
                                            G[path[y]][path[y+1]]['weight'], 
                                            G_backup[path[y]][path[y+1]]['weight'],
                                            towers_unlocked_iteration, 
                                            len(towers_unlocked_iteration) + len(path), 
                                            path]]
        G[path[y]][path[y+1]]['weight'] = 0

        
    #We create the data frame of this deployment's iteration and append it to the one with all deployments
    df_edges_prioritization_iteration = pd.DataFrame.from_records(path_iteration)
    df_edges_prioritization_iteration.columns = ['deployment_id', 'site', 'fiber_node', 'site_road', 'fiber_node_road', 'node_1', 'node_2', 'weight', 'old_weight', 'unlocked_sites', 'number_unlocked_sites', 'unlocked_nodes']

    df_edges_prioritization = df_edges_prioritization.append(df_edges_prioritization_iteration)

    #We calculate the ACTUAL density of this deployment (it may have changed due to previous deployments)
    density_iteration = df_prioritization_1_iteration['population'].iloc[0]/length
    density_iteration_old = df_prioritization_1_iteration['population'].iloc[0]/length_old
    
    population_iteration = df_prioritization_1_iteration['population'].iloc[0]
    
    #Unlocked sites and nodes in this iteration
    unlocked_sites_iteration = df_prioritization_1_iteration['unlocked_sites'].iloc[0].split(",")
    unlocked_nodes_iteration = df_prioritization_1_iteration['unlocked_nodes'].iloc[0].split(",")

    unlocked_sites = unlocked_sites + unlocked_sites_iteration
    
    #Add the current site to the prioritization list and its density to the density list
    prioritization_list = prioritization_list + [str(site_iteration)]
    prioritization_and_density = prioritization_and_density + [[str(int(tower_id_iteration)), density_iteration, density_iteration_old, population_iteration]]

    n_iteration = len(unlocked_sites_iteration)
    
    #Update the list of sites to deploy: we remove the sites unlocked (nodes and sites), as well
    df_prioritization_1 = df_prioritization_1.ix[~((df_prioritization_1['unlocked_site'].isin(unlocked_sites_iteration)) & (df_prioritization_1['unlocked_site'] != ''))]
    df_prioritization_1 = df_prioritization_1.ix[~((df_prioritization_1['unlocked_site_road'].isin(unlocked_nodes_iteration)) & (df_prioritization_1['unlocked_site'] != ''))]
    df_prioritization_1 = df_prioritization_1.ix[~df_prioritization_1['site'].isin(towers_unlocked_iteration)]
    df_prioritization_1 = df_prioritization_1.ix[~df_prioritization_1['site_road'].isin(unlocked_nodes_iteration)]
    df_prioritization_1 = df_prioritization_1.ix[df_prioritization_1['site'] != site_iteration]
    
df_prioritization_density = pd.DataFrame.from_records(prioritization_and_density, columns = ['site', 'population_density', 'population_density_unitary','unlocked_population'])
prioritization_list

In [ ]:
#Export new prioritization
pd.merge(df_edges_prioritization, df_prioritization_density, on = 'site').to_sql(table_output_2, con=engine, if_exists = 'replace', schema = schema, index = False, dtype={'unlocked_sites': sqlalchemy.types.VARCHAR})

query = """
UPDATE {schema}.{table_output_2} SET unlocked_sites = replace(unlocked_sites, '{{', '');
UPDATE {schema}.{table_output_2} SET unlocked_sites = replace(unlocked_sites, '}}', '');
UPDATE {schema}.{table_output_2} SET unlocked_nodes = replace(unlocked_nodes, '{{', '');
UPDATE {schema}.{table_output_2} SET unlocked_nodes = replace(unlocked_nodes, '}}', '');
"""

query_formatted = query.format(schema = schema,
                               schema_2 = schema_2,
                               table_output_2 = table_output_2).replace("[","").replace("]","")

connection = engine.connect()
engine.execute(query_formatted)

connection.close()

In [ ]:
#Check the missing sites that weren't deployed because of the weight update
#These chunks (from this one to export missing sites) need to be run several times until the query below 
#returns an empty table

connection = engine.connect()

query = """
SELECT DISTINCT ON (site, unlocked_site)
P.*,
C1.cluster_weight AS direct_population,
CASE WHEN C2.cluster_weight IS NULL THEN 0
     ELSE C2.cluster_weight END AS unlocked_population
FROM (
    
    SELECT 
    A.site,
    A.fiber_node,
    A.site_road,
    A.fiber_node_road,
    A.weight,
    A.unlocked_sites,
    A.number_unlocked_sites,
    A.unlocked_nodes,
    M.centroid AS unlocked_site,
    A.unlocked_site AS unlocked_site_road
    FROM (
        SELECT 
        site, 
        fiber_node,
        site_road,
        fiber_node_road,
        SUM(weight) AS weight,
        unlocked_sites,
        number_unlocked_sites,
        unlocked_nodes,
        CASE WHEN unlocked_nodes = '' THEN ''
             ELSE TRIM(UNNEST(string_to_array(REPLACE(unlocked_nodes,'',''), ','))) END AS unlocked_site
        FROM {schema}.{table_output_1}
        GROUP BY 1,2,3,4,6,7,unlocked_nodes
    )A
    LEFT JOIN {schema}.{table_cluster_node_map_los} M
    ON A.unlocked_site = M.node_id::text
    
    
    UNION 
    
    SELECT 
    site, 
    fiber_node,
    site_road,
    fiber_node_road,
    SUM(weight) AS weight,
    unlocked_sites,
    number_unlocked_sites,
    unlocked_nodes,
    CASE WHEN unlocked_sites = '' THEN ''
         ELSE TRIM(UNNEST(string_to_array(REPLACE(unlocked_sites,'',''), ','))) END AS unlocked_site,
    '' AS unlocked_site_road
    FROM {schema}.{table_output_1}
    GROUP BY 1,2,3,4,6,7,unlocked_nodes
    
    ORDER BY site
) P
LEFT JOIN {schema_2}.{table_clusters} C1
ON P.site = C1.centroid
LEFT JOIN {schema_2}.{table_clusters} C2
ON P.unlocked_site = C2.centroid
WHERE site IN (

    SELECT DISTINCT *
    FROM (

        SELECT DISTINCT
        site::text
        FROM {schema}.{table_output_1}

        UNION

        SELECT DISTINCT
        TRIM(UNNEST(string_to_array(REPLACE(unlocked_sites,'''',''), ',')))::text  AS site
        FROM {schema}.{table_output_1}
    )A
    WHERE site NOT IN (
    SELECT DISTINCT
    site::text
    FROM {schema}.{table_output_2}

    UNION

    SELECT DISTINCT
    TRIM(UNNEST(string_to_array(REPLACE(unlocked_sites,'''',''), ',')))::text  AS site
    FROM {schema}.{table_output_2}

    UNION


    SELECT 
    M.centroid AS unlocked_site FROM (
    SELECT 
    DISTINCT TRIM(UNNEST(string_to_array(REPLACE(unlocked_nodes,'',''), ','))) AS unlocked_site
    FROM {schema}.{table_output_2}
    ) P
    LEFT JOIN {schema}.{table_cluster_node_map} M
    ON P.unlocked_site = M.node_id::text
    WHERE M.node_id IS NOT NULL

    )

)
AND unlocked_site NOT IN (
    SELECT DISTINCT
    site::text
    FROM {schema}.{table_output_2}

    UNION

    SELECT DISTINCT
    TRIM(UNNEST(string_to_array(REPLACE(unlocked_sites,'''',''), ',')))::text  AS site
    FROM {schema}.{table_output_2}

    UNION


    SELECT 
    M.centroid AS unlocked_site FROM (
    SELECT 
    DISTINCT TRIM(UNNEST(string_to_array(REPLACE(unlocked_nodes,'',''), ','))) AS unlocked_site
    FROM {schema}.{table_output_2}
    ) P
    LEFT JOIN {schema}.{table_cluster_node_map} M
    ON P.unlocked_site = M.node_id::text
    WHERE M.node_id IS NOT NULL

)


UNION

SELECT 
site,
fiber_node,
site_road,
fiber_node_road,
weight,
unlocked_sites,
number_unlocked_sites,
unlocked_nodes,
'' AS unlocked_site,
'' AS unlocked_site_road,
direct_population,
0 AS unlocked_population
FROM 
(
        SELECT 
        P.*,
        C1.cluster_weight AS direct_population,
        CASE WHEN C2.cluster_weight IS NULL THEN 0
             ELSE C2.cluster_weight END AS unlocked_population
        FROM (
            
            SELECT 
            A.site,
            A.fiber_node,
            A.site_road,
            A.fiber_node_road,
            A.weight,
            A.unlocked_sites,
            A.number_unlocked_sites,
            A.unlocked_nodes,
            M.centroid AS unlocked_site
            FROM (
            SELECT 
            site, 
            fiber_node,
            site_road,
            fiber_node_road,
            SUM(weight) AS weight,
            unlocked_sites,
            number_unlocked_sites,
            unlocked_nodes,
            CASE WHEN unlocked_nodes = '' THEN ''
                 ELSE TRIM(UNNEST(string_to_array(REPLACE(unlocked_nodes,'',''), ','))) END AS unlocked_site
            FROM {schema}.{table_output_1} 
            GROUP BY 1,2,3,4,6,7,unlocked_nodes
            )A
            LEFT JOIN {schema}.{table_cluster_node_map} M
            ON A.unlocked_site = M.node_id::text
            
            
            UNION 
            
            SELECT 
            site, 
            fiber_node,
            site_road,
            fiber_node_road,
            SUM(weight) AS weight,
            unlocked_sites,
            number_unlocked_sites,
            unlocked_nodes,
            CASE WHEN unlocked_sites = '' THEN ''
                 ELSE TRIM(UNNEST(string_to_array(REPLACE(unlocked_sites,'',''), ','))) END AS unlocked_site
            FROM {schema}.{table_output_1}
            GROUP BY 1,2,3,4,6,7,unlocked_nodes
            
            ORDER BY site
        ) P
        LEFT JOIN {schema_2}.{table_clusters} C1
        ON P.site = C1.centroid
        LEFT JOIN {schema_2}.{table_clusters} C2
        ON P.unlocked_site = C2.centroid
) A
WHERE number_unlocked_sites > 0
AND site IN (

    SELECT DISTINCT *
    FROM (

    SELECT DISTINCT
    site::text
    FROM {schema}.{table_output_1}

    UNION

    SELECT DISTINCT
    TRIM(UNNEST(string_to_array(REPLACE(unlocked_sites,'''',''), ',')))::text  AS site
    FROM {schema}.{table_output_1}
    )A
    WHERE site NOT IN (
    SELECT DISTINCT
    site::text
    FROM {schema}.{table_output_2}

    UNION

    SELECT DISTINCT
    TRIM(UNNEST(string_to_array(REPLACE(unlocked_sites,'''',''), ',')))::text  AS site
    FROM {schema}.{table_output_2}

    UNION


    SELECT 
    M.centroid AS unlocked_site FROM (
    SELECT 
    DISTINCT TRIM(UNNEST(string_to_array(REPLACE(unlocked_nodes,'',''), ','))) AS unlocked_site
    FROM {schema}.{table_output_2}
    ) P
    LEFT JOIN {schema}.{table_cluster_node_map} M
    ON P.unlocked_site = M.node_id::text
    WHERE M.node_id IS NOT NULL

    )

)

ORDER BY site, unlocked_site, unlocked_site_road


"""
    
    

query_formatted = query.format(schema = schema,
                               schema_2 = schema_2,
                               table_nodes_roads = table_nodes_roads,
                               table_clusters = table_clusters,
                               table_output_1 = table_output_1,
                               table_cluster_node_map = table_cluster_node_map,
                               table_cluster_node_map_los = table_cluster_node_map_los,
                               table_output_2 = table_output_2).replace("[","").replace("]","")
connection = engine.connect()

df_missing = pd.read_sql_query(query_formatted, con = engine)
df_missing = df_missing.ix[df_missing['site'].isin(list(df_towers_to_connect_fiber['centroid']))]

connection.close()

missing_list = list(df_missing['site'])
df_missing['fiber_node'] = df_missing['fiber_node'].astype(float).astype(int)

df_missing
#missing_list
#print(query_formatted)

In [ ]:
#We repeat this a number of times to make sure we account for everything. We will skip as soon as it is finished
n_iterations = 20
for x in range(0, n_iterations):
    print(str(x)  + ' / ' + str(n_iterations))
    #We load missing sites
    connection = engine.connect()

    df_missing = pd.read_sql_query(query_formatted, con = engine)
    df_missing = df_missing.ix[df_missing['site'].isin(list(df_towers_to_connect_fiber['centroid']))]
    df_missing['fiber_node'] = df_missing['fiber_node'].astype(float).astype(int)
    
    connection.close()
    
    #If it is empty, we finish
    if(df_missing.empty):
        break

    

    missing_list = list(df_missing['site'])
    
    #Prioritization - deploy sites that were left out  (same logic as the regular prioritization)
    prioritization_list_missing = []
    unlocked_sites = []
    prioritization_and_density = []
    x = -1

    df_towers_to_connect_fiber = df_towers_to_connect_fiber_backup.copy(deep = True)
    df_edges_prioritization = pd.DataFrame(columns = ['deployment_id', 'site', 'fiber_node', 'site_road', 'fiber_node_road', 'node_1', 'node_2', 'weight', 'old_weight','unlocked_sites', 'number_unlocked_sites', 'unlocked_nodes'])

    while(not(df_missing.empty)):
        x = x + 1
        if(x%10 == 0):
            print(x)

        #Data frame for current iteration
        df_prioritization_1_iteration = df_missing.copy(deep = True)
        df_prioritization_1_iteration = df_prioritization_1_iteration.fillna(0)

        #We calculate the population unlocked for each deployment (direct and indirect) as well as the population density for each site
        df_prioritization_1_iteration = df_prioritization_1_iteration.groupby('site').sum().reset_index()
        df_prioritization_1_iteration = pd.merge(df_missing, df_prioritization_1_iteration[['site', 'unlocked_population']], how = 'right', on = 'site')
        df_prioritization_1_iteration = df_prioritization_1_iteration.drop('unlocked_population_x', axis = 1)
        df_prioritization_1_iteration = df_prioritization_1_iteration.rename(index=str, columns={"unlocked_population_y": "unlocked_population"})

        df_prioritization_1_iteration['population'] = df_prioritization_1_iteration['direct_population']+df_prioritization_1_iteration['unlocked_population']
        df_prioritization_1_iteration['population'] = df_prioritization_1_iteration['unlocked_population']
        df_prioritization_1_iteration['population_density'] = df_prioritization_1_iteration['population']/df_prioritization_1_iteration['weight']

        df_prioritization_1_iteration = df_prioritization_1_iteration.drop_duplicates('site').sort_values('population_density', ascending = False).reset_index(drop = True)


        #We select the most dense deployment and deploy it (it is not actually the most dense. We still do not account for previous deployments)

        #Current site to deploy, fiber node to use as source and density of the deployment (in two references: graph-wise and Rural Planner-wise)
        tower_id_iteration = df_prioritization_1_iteration['site'].iloc[0]
        fiber_node_iteration = str(int(df_prioritization_1_iteration['fiber_node_road'].iloc[0]))

        site_iteration = str(int(df_prioritization_1_iteration['site_road'].iloc[0]))
        fiber_tower_iteration = str(int(df_prioritization_1_iteration['fiber_node'].iloc[0]))

        #Calculate shortest path for current node and its length
        path = nx.dijkstra_path(G, site_iteration, fiber_node_iteration)
        length = nx.dijkstra_path_length(G, site_iteration, fiber_node_iteration)
        length_old = nx.dijkstra_path_length(G_backup, site_iteration, fiber_node_iteration)

        #Check if we unlocked any intermediate tower in the way we deployed the fiber (now it only checks towers in the set of towers to connect)
        towers_unlocked_iteration_fiber = df_towers_to_connect_fiber.ix[(df_towers_to_connect_fiber['node_id'].astype(int).astype(str).isin(path)) & 
                                                        (df_towers_to_connect_fiber['node_id'].astype(int).astype(str) != site_iteration)]['centroid'].values.tolist()

        #Check if connecting this site unlocks any other sites (belonging to the set of sites we want to connect)
        if(df_line_of_sight.ix[((df_line_of_sight['tower_id_1'] == int(tower_id_iteration)))].empty):
            towers_unlocked_1_iteration = []
        else:
            towers_unlocked_1_iteration = df_line_of_sight.ix[((df_line_of_sight['tower_id_1'] == int(tower_id_iteration)))]['tower_id_2'].values.tolist()

        if(df_line_of_sight.ix[((df_line_of_sight['tower_id_2'] == int(tower_id_iteration)))].empty):
            towers_unlocked_2_iteration = []
        else:
            towers_unlocked_2_iteration = df_line_of_sight.ix[((df_line_of_sight['tower_id_2'] == int(tower_id_iteration)))]['tower_id_1'].values.tolist()

        #Set of all unlocked sites
        list_towers = [str(x) for x in list(towers_unlocked_1_iteration + towers_unlocked_2_iteration + towers_unlocked_iteration_fiber)]
        #list_towers = [str(x) for x in list(towers_unlocked_1_iteration + towers_unlocked_2_iteration)]
        towers_unlocked_iteration = list_towers

        #We create the data frame with all the edges needed to reach the current site and update the weight of the edges used to zero
        path_iteration = []
        for y in range(0, len(path)-1):
            path_iteration = path_iteration + [[x + len(prioritization_list), #+ 21 + 1, 
                                                tower_id_iteration, 
                                                fiber_tower_iteration, 
                                                site_iteration, 
                                                fiber_node_iteration, 
                                                path[y], 
                                                path[y+1], 
                                                G[path[y]][path[y+1]]['weight'], 
                                                G_backup[path[y]][path[y+1]]['weight'], 
                                                towers_unlocked_iteration, 
                                                len(towers_unlocked_iteration) + len(path), 
                                                path]]
            G[path[y]][path[y+1]]['weight'] = 0


        #We create the data frame of this deployment's iteration and append it to the one with all deployments
        df_edges_prioritization_iteration = pd.DataFrame.from_records(path_iteration)
        df_edges_prioritization_iteration.columns = ['deployment_id', 'site', 'fiber_node', 'site_road', 'fiber_node_road', 'node_1', 'node_2', 'weight', 'old_weight', 'unlocked_sites', 'number_unlocked_sites', 'unlocked_nodes']

        df_edges_prioritization = df_edges_prioritization.append(df_edges_prioritization_iteration)

        #We calculate the ACTUAL density of this deployment
        density_iteration = df_prioritization_1_iteration['population'].iloc[0]/length
        density_iteration_old = df_prioritization_1_iteration['population'].iloc[0]/length_old

        population_iteration = df_prioritization_1_iteration['population'].iloc[0]

        #Unlocked sites and nodes in this iteration
        unlocked_sites_iteration = df_prioritization_1_iteration['unlocked_sites'].iloc[0].split(",")
        unlocked_nodes_iteration = df_prioritization_1_iteration['unlocked_nodes'].iloc[0].split(",")

        unlocked_sites = unlocked_sites + unlocked_sites_iteration

        #Add the current site to the prioritization list and its density to the density list
        #prioritization_list = prioritization_list + [str(site_iteration)]
        prioritization_and_density = prioritization_and_density + [[str(int(tower_id_iteration)), density_iteration, density_iteration_old, population_iteration]]

        n_iteration = len(unlocked_sites_iteration)


        #Update the list of sites to deploy: we remove the sites unlocked (nodes and sites), as well
        df_missing = df_missing.ix[~((df_missing['unlocked_site'].isin(unlocked_sites_iteration)) & (df_missing['unlocked_site'] != ''))]
        df_missing = df_missing.ix[~((df_missing['unlocked_site_road'].isin(unlocked_nodes_iteration)) & (df_missing['unlocked_site'] != ''))]
        df_missing = df_missing.ix[~df_missing['site'].isin(towers_unlocked_iteration)]
        df_missing = df_missing.ix[~df_missing['site_road'].isin(unlocked_nodes_iteration)]
        df_missing = df_missing.ix[df_missing['site'] != site_iteration]

    df_prioritization_density = pd.DataFrame.from_records(prioritization_and_density, columns = ['site', 'population_density', 'population_density_unitary','unlocked_population'])
    
    #Append the final deployments
    pd.merge(df_edges_prioritization, df_prioritization_density, on = 'site').to_sql(table_output_2, con=engine, if_exists = 'append', schema = schema, index = False, dtype={'unlocked_sites': sqlalchemy.types.VARCHAR})

    query_update = """
    UPDATE {schema}.{table_output_2} SET unlocked_sites = replace(unlocked_sites, '{{', '');
    UPDATE {schema}.{table_output_2} SET unlocked_sites = replace(unlocked_sites, '}}', '');
    UPDATE {schema}.{table_output_2} SET unlocked_nodes = replace(unlocked_nodes, '{{', '');
    UPDATE {schema}.{table_output_2} SET unlocked_nodes = replace(unlocked_nodes, '}}', '');
    """
    query_formatted_update = query_update.format(schema = schema,
                                   schema_2 = schema_2,
                                   table_output_2 = table_output_2).replace("[","").replace("]","")

    connection = engine.connect()
    engine.execute(query_formatted_update)

    connection.close()
    

In [ ]:
#Cropping process (we remove all non-profitable deployments)
#Load every fiber node with their deployments

query = """

SELECT 
fiber_node,
deployment_id,
site,
SUM(weight) AS weight,
SUM(old_weight) AS old_weight,
unlocked_sites,
unlocked_nodes,
population_density,
unlocked_population
FROM {schema}.{table_output_2}
GROUP BY fiber_node, deployment_id, site, unlocked_sites, unlocked_nodes, population_density, unlocked_population
ORDER BY fiber_node, deployment_id DESC

"""
query_formatted = query.format(schema = schema,
                               schema_2 = schema_2,
                               table_nodes_roads = table_nodes_roads,
                               table_clusters = table_clusters,
                               table_output_1 = table_output_1,
                               table_cluster_node_map = table_cluster_node_map,
                               table_output_2 = table_output_2).replace("[","").replace("]","")
connection = engine.connect()

df_nodes_crop = pd.read_sql_query(query_formatted, con = engine)

connection.close()
#Crop nodes

#List of all fiber nodes to check
nodes = list(set(list(df_nodes_crop['fiber_node'].values)))
n_nodes = len(nodes)
#n_nodes = 20

#profitability threshold - This is the minimum people to cover with a 1-km deployment
threshold = 214.29 

list_sites_to_crop = {}

#For each FIBER NODE, we calculate its global profitability. We then crop all non-profitable leaves (i.e. deployments) 
#until either there are only profitable leaves or 
#there are no more branches. Then we re-calculate the new profitability and, if the cropped node is not profitable, then we deploy only the sites
#that are profitable individually. If there are none, we crop the whole node.

#Loop through all FIBER NODES
for x in range(0, n_nodes):
    
    #Current node. We sort by descending deployment so that we crop leaves and not branches
    list_sites_to_crop_iteration = []
    list_sites_profitable = []
    list_nodes_to_reach_profitable_sites = []
    node_iteration = nodes[x]
    
    df_node_iteration = df_nodes_crop.ix[df_nodes_crop['fiber_node'] == node_iteration].sort_values('deployment_id', ascending = False).reset_index(drop = True)
    df_node_iteration['old_population_density'] = df_node_iteration['unlocked_population']/df_node_iteration['old_weight']
    
    n_iterations = len(df_node_iteration['deployment_id'])
    
    df_node_iteration_aggregate = df_node_iteration[['fiber_node', 'weight', 'unlocked_population']].groupby('fiber_node').sum().reset_index()
    df_node_iteration_aggregate['population_density'] = df_node_iteration_aggregate['unlocked_population']/df_node_iteration_aggregate['weight']
    
    node_iteration_density = df_node_iteration_aggregate['population_density'].iloc[0]
    
    
    #We calculate the profitability of the node as a whole
    if(node_iteration_density >= threshold):
        profitability_iteration = True
    else:
        profitability_iteration = False 
    
    #We crop the non-profitable LEAVES (external nodes)
    for y in range(0, n_iterations):
        
        if(df_node_iteration['population_density'].iloc[y] < threshold):
            
            list_nodes_unlocked_iteration = df_node_iteration['unlocked_nodes'].iloc[y]
            list_sites_dependent = []
            
            #We check which of the previous profitable sites depend on the current non-profitable site
            for z in range(0, len(list_sites_profitable)):
                
                list_nodes_unlocked_to_compare_iteration = list_nodes_to_reach_profitable_sites[z][1]
                site_to_compare_iteration = list_nodes_to_reach_profitable_sites[z][0]
                
                list_common_elements_1 = set(str(set([list_nodes_unlocked_iteration])).replace("{", "").replace("}", "").replace("'", "").split(","))
                list_common_elements_2 = set(str(set([list_nodes_unlocked_to_compare_iteration])).replace("{", "").replace("}", "").replace("'", "").split(","))
                n_common_elements = len(list_common_elements_1.intersection(list_common_elements_2))
                if(n_common_elements/float(max(len(list_common_elements_1), len(list_common_elements_2))) > 0.5):
                    list_sites_dependent = list_sites_dependent + [site_to_compare_iteration]
            
            #We filter data frame with all profitable leaves dependant on the current leave
            df_profitable_node_iteration_aggregate = df_node_iteration.ix[df_node_iteration['site'].isin(list_sites_dependent)][['fiber_node', 'weight', 'unlocked_population']].groupby('fiber_node').sum().reset_index()
            
            #If there are none, we leave the profitability as is
            if(df_profitable_node_iteration_aggregate.empty):
                profitable_node_iteration_density = df_node_iteration['population_density'].iloc[y]
            
            #Otherwise, we update it as the compound profitability of all dependant leaves and the current leave
            else:
                df_profitable_node_iteration_aggregate['population_density'] = df_profitable_node_iteration_aggregate['unlocked_population']/df_profitable_node_iteration_aggregate['weight']
                profitable_node_iteration_density = df_profitable_node_iteration_aggregate['population_density'].iloc[0]
            
            if(profitable_node_iteration_density < threshold):
                list_sites_to_crop_iteration = list_sites_to_crop_iteration + [df_node_iteration['site'].iloc[y]]
            else:
                list_sites_to_crop_iteration = list_sites_to_crop_iteration
            
        else:
            list_sites_profitable = list_sites_profitable + [df_node_iteration['site'].iloc[y]]
            list_nodes_to_reach_profitable_sites = list_nodes_to_reach_profitable_sites + [(df_node_iteration['site'].iloc[y], df_node_iteration['unlocked_nodes'].iloc[y])]
            
    
    #We remove the cropped sites from the node
    df_node_iteration = df_node_iteration.ix[~df_node_iteration['site'].isin(list_sites_to_crop_iteration)]
    
    #If there are no sites left, we exit
    if(df_node_iteration.empty):
        [str(x) for x in list_sites_to_crop_iteration]
        list_sites_to_crop_iteration = ",".join(list_sites_to_crop_iteration)
        list_sites_to_crop[node_iteration] = list_sites_to_crop_iteration
        continue
    
    #Otherwise, leave only those that were profitable independently of the prioritization
    df_node_iteration_aggregate_new = df_node_iteration[['fiber_node', 'weight', 'unlocked_population']].groupby('fiber_node').sum().reset_index()
    df_node_iteration_aggregate_new['population_density'] = df_node_iteration_aggregate_new['unlocked_population']/df_node_iteration_aggregate_new['weight']
    
    node_iteration_density = df_node_iteration_aggregate_new['population_density'].iloc[0]
    
    if(node_iteration_density >= threshold):
        profitability_iteration = True
    else:
        profitability_iteration = False
    
    #If the node is profitable, we leave it as is
    if(profitability_iteration):
        [str(x) for x in list_sites_to_crop_iteration]
        list_sites_to_crop_iteration = ",".join(list_sites_to_crop_iteration)
        list_sites_to_crop[node_iteration] = list_sites_to_crop_iteration
        continue
    
    #Otherwise, we deploy only the individual profitable paths 
    print(node_iteration)
    remaining_sites_to_crop = list(df_node_iteration.ix[df_node_iteration['old_population_density'] < threshold, 'site'])

    list_sites_to_crop_iteration = list_sites_to_crop_iteration + remaining_sites_to_crop
    
    [str(x) for x in list_sites_to_crop_iteration]
    list_sites_to_crop_iteration = ",".join(list_sites_to_crop_iteration)
    list_sites_to_crop[node_iteration] = list_sites_to_crop_iteration

list_sites_to_crop

df_crop_output = pd.DataFrame.from_dict(list_sites_to_crop, 'index', dtype = str)
df_crop_output['fiber_node'] = df_crop_output.index
df_crop_output.columns = ['nodes_to_crop', 'fiber_node']
df_crop_output = df_crop_output[['fiber_node', 'nodes_to_crop']]

df_crop_output

df_crop_output.to_sql(table_output_crop, con=engine, if_exists = 'replace', schema = schema, index = False)

In [ ]:
#Perform second Microwave analysis with the extended network

#Load edges
query_path = sql_path + '/' + country_folder + '/' + 'path_finder_mw_load_edges.sql'

with open(query_path) as file, database(parser) as db:
    query = file.read()
    query_formatted = query.format(schema = schema, table_line_of_sight = table_line_of_sight, table_towers = table_towers)
    df_edges = pd.read_sql_query(query_formatted, con = db)
    
#Create graph
Gm = nx.Graph()

for z in range(0, len(df_edges)):
    Gm.add_edge(str(int(df_edges['tower_id_1'].iloc[z])),
               str(int(df_edges['tower_id_2'].iloc[z])))
    Gm.add_edge(str(int(df_edges['tower_id_2'].iloc[z])),
               str(int(df_edges['tower_id_1'].iloc[z])))
    
graphs = list(nx.connected_component_subgraphs(Gm))

In [ ]:
#Load fiber nodes or nodes deployed in the previous process

engine = create_engine(conn_string)

#Loads all fiber nodes and nodes contained in any fiber deployment (direct or indirect) and not cropped
query = """
SELECT 
*
FROM {schema_2}.{table_towers} I1
WHERE fiber IS TRUE
OR tower_id::text IN (
        SELECT DISTINCT *
        FROM (
        
            
            SELECT 
            DISTINCT TRIM(UNNEST(string_to_array(REPLACE(unlocked_sites,'',''), ',')))::text AS unlocked_site
            FROM {schema}.{table_output_2}
            WHERE site NOT IN (
                SELECT
                TRIM(UNNEST(string_to_array(REPLACE(nodes_to_crop,'',''), ','))) AS site_cropped
                FROM {schema}.{table_output_crop}
            )

            UNION
        
            SELECT DISTINCT
            site::text
            FROM {schema}.{table_output_2}
            WHERE site NOT IN (
                SELECT
                TRIM(UNNEST(string_to_array(REPLACE(nodes_to_crop,'',''), ','))) AS site_cropped
                FROM {schema}.{table_output_crop}
            )

        )A

)
"""



query_formatted = query.format(schema = schema,
                               schema_2 = schema_2,
                               table_towers = table_towers,
                               table_output_2 = table_output_2,
                               table_cluster_node_map = table_cluster_node_map,
                               table_output_crop = table_output_crop,
                               table_line_of_sight = table_line_of_sight).replace("[","").replace("]","")
connection = engine.connect()

df_edges_fiber = pd.read_sql_query(query_formatted, con = engine)

#Load terminal nodes


#Load nodes that needed to be connected and that didn't find any solution in the FO part
query = """
SELECT *
FROM {schema_2}.{table_towers}
WHERE ipt_perimeter = 'IPT'
AND tech_3g IS FALSE
AND tech_4g IS FALSE
AND radio IS FALSE 
AND fiber IS FALSE
AND tower_id::text NOT IN (
        SELECT DISTINCT *
        FROM (
        
            
            SELECT 
            DISTINCT TRIM(UNNEST(string_to_array(REPLACE(unlocked_sites,'',''), ',')))::text AS unlocked_site
            FROM {schema}.{table_output_2}
            WHERE site NOT IN (
                SELECT
                TRIM(UNNEST(string_to_array(REPLACE(nodes_to_crop,'',''), ','))) AS site_cropped
                FROM {schema}.{table_output_crop}
            )

            UNION
        
            SELECT DISTINCT
            site::text
            FROM {schema}.{table_output_2}
            WHERE site NOT IN (
                SELECT
                TRIM(UNNEST(string_to_array(REPLACE(nodes_to_crop,'',''), ','))) AS site_cropped
                FROM {schema}.{table_output_crop}
            )

        )A

)
"""

query_formatted = query.format(schema = schema,
                               schema_2 = schema_2,
                               table_towers = table_towers,
                               table_output_2 = table_output_2,
                               table_output_crop = table_output_crop,
                               table_line_of_sight = table_line_of_sight).replace("[","").replace("]","")
connection = engine.connect()

df_edges_terminal = pd.read_sql_query(query_formatted, con = engine)

connection.close()

In [ ]:
#Calculate mw paths
print(str(datetime.now()))
all_terminal_nodes = df_edges_terminal['tower_id'].astype(int).astype(str).tolist()

df_output = pd.DataFrame()

#One iteration for every connected independent subgraph
for k in range(0, len(graphs)):
    
    #Take one of the connected subgraphs and turn it into a data frame
    G1 = graphs[k]

    nodes_subgraph = list(G1.nodes())
    
        
    #This is the data frame with the edges associated to the fiber
    df_edges_fiber_all = df_edges_fiber.ix[df_edges_fiber['tower_id'].isin(nodes_subgraph),]    
    
    # Filter terminal nodes from current subgraph 
    terminal_nodes = { node for node in all_terminal_nodes if node in nodes_subgraph}
    terminal_nodes = list(terminal_nodes)

    if len(terminal_nodes)>0:
        #Calculate optimal path for every terminal node with Dijkstra's algorithm
        df_output_subgraph = pd.DataFrame({'node_id': terminal_nodes})

        for owner in owners.keys():    
            df_edges_fiber_owner =  df_edges_fiber_all.ix[df_edges_fiber_all['source'].isin(owners[owner]['sources'])]
            fiber_nodes_owner = df_edges_fiber_owner['tower_id'].astype(int).astype(str).tolist()

            # calculate optimal fiber path per owner
            df_output_subgraph = df_output_subgraph.join(df_output_subgraph.progress_apply(solve_mw_path, axis=1, args=(owner, fiber_nodes_owner, G1, df_edges)))
            print(str(owner)+ ' processed.')   

        df_output = df_output.append(df_output_subgraph)     
        del df_output_subgraph

In [ ]:
output_cols = list(['node_id'])

for owner in owners.keys():
    owners_cols = list(['length_' + owner,
                   'path_' + owner, 
                   'fiber_node_' + owner])
    output_cols.extend(owners_cols)
    
sort_cols = ['node_id']
sort_cols.extend(['fiber_node_'+owner for owner in owners.keys()])

df_output = df_output[output_cols].sort_values(sort_cols).drop_duplicates('node_id').reset_index(drop = True)

In [ ]:
##export, replace arrays limits, cast variables to int/float, create geoms

query_path_export = sql_path + '/' + country_folder + '/' + 'path_finder_mw_export.sql'
query_path_final = sql_path + '/' + country_folder + '/' + 'path_finder_mw_final_table.sql'

with open(query_path_export) as file_export, open(query_path_final) as file_final, database(parser) as db:
    df_output.to_sql(table_output_microwave, con=db, if_exists = 'replace', schema = schema, index = False)
    query = file_export.read()
    query_formatted = query.format(schema = schema,
                                   output_table = output_table,
                                   table_towers = table_towers)
    db.execute(query_formatted)
    # Create final output
    query = file_final.read()
    query_formatted = query.format(schema = schema,
                                   final_table = table_output_microwave_2,
                                   table_clusters = table_clusters,
                                   table_towers = table_towers,
                                   output_table = table_output_microwave)
    db.execute(query_formatted)

In [ ]:
#Export quick wins as well and create geom table


#Export quick wins
df_towers_quick_wins_fiber.to_sql(table_quick_wins_fiber, con=engine, if_exists = 'replace', schema = schema, index = False)
df_towers_quick_wins_radio.to_sql(table_quick_wins_radio, con=engine, if_exists = 'replace', schema = schema, index = False)


#Create geometry table
query = """
DROP TABLE IF EXISTS {schema}.{table_geom_fiber};
CREATE TABLE {schema}.{table_geom_fiber} AS
SELECT ST_Union(geom) AS geom
FROM (
SELECT 
E.*,
ST_MakeLine(N1.geom::geometry, N2.geom::geometry) AS geom
FROM {schema}.{table_output_2} E
LEFT JOIN (
    SELECT
    node_id,
    geom::geography
    FROM {schema}.{table_nodes} 

    UNION 

    SELECT
    tower_id + (SELECT MAX(node_id::numeric) FROM {schema}.{table_nodes}) AS node_id,
    geom
    FROM {schema_2}.{table_towers}
    WHERE fiber IS TRUE
)N1
ON E.node_1::text = N1.node_id::text
LEFT JOIN (
    SELECT 
    node_id,
    geom::geography
    FROM {schema}.{table_nodes}

    UNION 

    SELECT
    tower_id + (SELECT MAX(node_id::numeric) FROM {schema}.{table_nodes}) AS node_id,
    geom
    FROM {schema_2}.{table_towers}
    WHERE fiber IS TRUE
)N2
ON E.node_2::text = N2.node_id::text
) A
WHERE site NOT IN (
    SELECT
    TRIM(UNNEST(string_to_array(REPLACE(nodes_to_crop,'',''), ','))) AS site_cropped
    FROM {schema}.{table_output_crop}
)
"""
query_formatted = query.format(schema = schema,
                               schema_2 = schema_2,
                               table_output_microwave = table_output_microwave,
                               table_towers = table_towers,
                               table_output_microwave_2 = table_output_microwave_2,
                               table_output_2 = table_output_2,
                               table_geom_fiber = table_geom_fiber,
                               table_output_basic = table_output_basic,
                               table_quick_wins_radio = table_quick_wins_radio,
                               table_quick_wins_fiber = table_quick_wins_fiber,
                               table_output_crop = table_output_crop,
                               table_nodes = table_nodes).replace("[","").replace("]","")
connection = engine.connect()


engine.execute(query_formatted)


connection.close()

In [ ]:
#Create first basic version of the final output

query = """
DROP TABLE IF EXISTS {schema}.{table_output_basic};
CREATE TABLE {schema}.{table_output_basic} AS

SELECT 
id_despliegue,
segmento,
sitio,
nodo_fibra,
longitud_fibra_km,
saltos_radioenlace,
camino_radioenlaces,
geom_fibra,
geom_radioenlaces
FROM (

SELECT DISTINCT ON (sitio)
*
FROM (

--Radioenlaces - Quick wins
SELECT DISTINCT ON (sitio)
-1 AS id_despliegue,
0 AS priority,
'MW QUICK WINS' AS segmento,
MWQW.centroid AS sitio,
CASE WHEN hops_movistar <= 1 THEN fiber_node_movistar
     WHEN hops_azteca <= 1 THEN fiber_node_azteca
     WHEN hops_third_party <= 1 THEN fiber_node_third_party END AS nodo_fibra,
NULL::numeric AS longitud_fibra_km,
CASE WHEN hops_movistar <= 1 THEN hops_movistar
     WHEN hops_azteca <= 1 THEN hops_azteca
     WHEN hops_third_party <= 1 THEN hops_third_party END AS saltos_radioenlace,
CASE WHEN hops_movistar <= 1 THEN path_movistar
     WHEN hops_azteca <= 1 THEN path_azteca
     WHEN hops_third_party <= 1 THEN path_third_party END AS camino_radioenlaces,
NULL::geometry AS geom_fibra,
G.geom AS geom_radioenlaces
FROM {schema}.{table_quick_wins_radio} MWQW
LEFT JOIN (
        SELECT 
        U1.centroid,
        ST_Union(ST_MakeLine(U1.geom, U2.geom)) AS geom
        FROM (
        
        SELECT
        row_number,
        A.centroid,
        I.tower_id,
        I.geom::geometry
        FROM (
                SELECT 
                row_number() OVER () AS row_number,
                *
                FROM (
                        SELECT
                        centroid,
                        CASE WHEN hops_movistar <= 1 THEN TRIM(UNNEST(string_to_array(REPLACE(path_movistar,'',''), ',')))
                             WHEN hops_azteca <= 1 THEN TRIM(UNNEST(string_to_array(REPLACE(path_movistar,'',''), ',')))
                             WHEN hops_third_party <= 1 THEN TRIM(UNNEST(string_to_array(REPLACE(path_movistar,'',''), ','))) END AS salto
                        FROM {schema}.{table_quick_wins_radio} MWQW
                ) X
        ) A
        LEFT JOIN {schema_2}.{table_towers} I
        ON I.tower_id::text = A.salto
        ORDER BY row_number
        ) U1
        LEFT JOIN (
        
        
        SELECT
        row_number,
        A.centroid,
        I.tower_id,
        I.geom::geometry
        FROM (
                SELECT 
                row_number() OVER () AS row_number,
                *
                FROM (
                        SELECT
                        centroid,
                        CASE WHEN hops_movistar <= 1 THEN TRIM(UNNEST(string_to_array(REPLACE(path_movistar,'',''), ',')))
                             WHEN hops_azteca <= 1 THEN TRIM(UNNEST(string_to_array(REPLACE(path_movistar,'',''), ',')))
                             WHEN hops_third_party <= 1 THEN TRIM(UNNEST(string_to_array(REPLACE(path_movistar,'',''), ','))) END AS salto
                        FROM {schema}.{table_quick_wins_radio} MWQW
                ) X
        ) A
        LEFT JOIN {schema_2}.{table_towers} I
        ON I.tower_id::text = A.salto
        ORDER BY row_number
        ) U2
        ON (U1.centroid = U2.centroid AND U1.row_number = U2.row_number  + 1)
        GROUP BY U1.centroid
        
) G
ON MWQW.centroid = G.centroid


UNION

--Despliegues directos de fibra - Quick wins
SELECT DISTINCT ON (deployment_id)
deployment_id AS id_despliegue,
1 AS priority,
'FO QUICK WINS' AS segmento,
site AS sitio,
fiber_node AS nodo_fibra,
SUM(weight) AS longitud_fibra_km,
NULL::numeric AS saltos_radioenlace,
NULL::text AS camino_radioenlaces,
NULL::geometry AS geom_fiber,
NULL::geometry AS geom_radioenlaces
FROM {schema}.{table_output_2}
WHERE site IN (
        SELECT 
        centroid
        FROM {schema}.{table_quick_wins_fiber}
)
AND site NOT IN (
    SELECT
    TRIM(UNNEST(string_to_array(REPLACE(nodes_to_crop,'',''), ','))) AS site_cropped
    FROM {schema}.{table_output_crop}
)
GROUP BY deployment_id, site, fiber_node, population_density


UNION

--Quick Wins iniciales que no se resolviesen con N saltos de radioenlace (cerca de un POP de FO pero sin LoS)

SELECT DISTINCT ON (centroid)
-1 AS id_despliegue,
1 AS priority,
'NEAR FO POP QUICK WINS' AS segmento,
centroid::text AS sitio,
fiber_node::text AS nodo_fibra,
distance_to_fiber*1.5/1000 AS longitud_fibra_km,
NULL::numeric AS saltos_radioenlace,
NULL::text AS camino_radioenlaces,
geom_line::geometry AS geom_fibra,
NULL::geometry AS geom_radioenlaces
FROM {schema}.{table_initial_quick_wins} FOQW2
WHERE centroid::text IN (
        SELECT centroid
        FROM {schema}.{table_output_microwave_2}
        WHERE hops_movistar IS NOT NULL OR hops_azteca IS NOT NULL OR hops_regional IS NOT NULL OR hops_third_party IS NOT NULL
)
AND fiber_node IN (
        SELECT tower_id
        FROM {schema_2}.{table_towers}
        WHERE fiber IS TRUE
)



UNION

--Quick Wins iniciales que no se resolviesen con N saltos de radioenlace (cerca de un POP de MW pero sin LoS)
SELECT DISTINCT ON (centroid)
-1 AS id_despliegue,
1 AS priority,
'NEAR MW POP QUICK WINS' AS segmento,
centroid::text AS sitio,
fiber_node::text AS nodo_fibra,
distance_to_fiber*1.5/1000 AS longitud_fibra_km,
2 AS saltos_radioenlace,
centroid::text || ', ' || fiber_node::text AS camino_radioenlaces,
NULL::geometry AS geom_fibra,
geom_line::geometry AS geom_radioenlaces
FROM {schema}.{table_initial_quick_wins}
WHERE centroid::text IN (
        SELECT centroid
        FROM {schema}.{table_output_microwave_2}
        WHERE hops_movistar IS NOT NULL OR hops_azteca IS NOT NULL OR hops_regional IS NOT NULL OR hops_third_party IS NOT NULL
)
AND fiber_node NOT IN (
        SELECT tower_id
        FROM {schema_2}.{table_towers}
        WHERE fiber IS TRUE
)


UNION


--Sitios desbloqueados con Fibra
SELECT *
FROM (

SELECT DISTINCT ON (TRIM(UNNEST(string_to_array(REPLACE(unlocked_sites,'''',''), ',')))::text)
deployment_id AS id_despliegue,
1 AS priority,
'FO INDIRECT UNLOCKED'::text AS segmento,
TRIM(UNNEST(string_to_array(REPLACE(unlocked_sites,'''',''), ',')))::text  AS sitio,
fiber_node AS nodo_fibra,
0 AS longitud_fibra_km,
NULL::numeric AS saltos_radioenlace,
NULL::text AS camino_radioenlaces,
NULL::geometry AS geom_fibra,
NULL::geometry AS geom_radioenlaces
FROM {schema}.{table_output_2}
WHERE site NOT IN (
    SELECT
    TRIM(UNNEST(string_to_array(REPLACE(nodes_to_crop,'',''), ','))) AS site_cropped
    FROM {schema}.{table_output_crop}
)
ORDER BY sitio, id_despliegue
) A

WHERE sitio IN (
        SELECT centroid::text
        FROM {schema}.{table_output_2} P
        LEFT JOIN {schema}.{table_cluster_node_map_los} C
        ON P.node_2 = C.node_id::text
        WHERE site NOT IN (
            SELECT
            TRIM(UNNEST(string_to_array(REPLACE(nodes_to_crop,'',''), ','))) AS site_cropped
            FROM {schema}.{table_output_crop}
        )
)
AND sitio IN (
        SELECT DISTINCT centroid
        FROM {schema}.{table_cluster_node_map_los}
        WHERE ST_Distance(geom_node::geography, geom_centroid::geography) <= 1000
)


UNION

--Sitios desbloqueados con Radioenlace 
SELECT DISTINCT ON (sitio)
*
FROM (
--desde el sitio final que se desplegaba

SELECT 
id_despliegue,
priority,
segmento,
sitio,
nodo_fibra,
longitud_fibra_km,
saltos_radioenlace,
sitio || ', ' || site AS camino_radioenlaces,
geom_fibra,
ST_MakeLine(I1.geom::geometry, I2.geom::geometry) AS geom_radioenlaces
FROM (

SELECT DISTINCT ON (TRIM(UNNEST(string_to_array(REPLACE(unlocked_sites,'''',''), ',')))::text)
site,
deployment_id AS id_despliegue,
1 AS priority,
'MW INDIRECT UNLOCKED'::text AS segmento,
TRIM(UNNEST(string_to_array(REPLACE(unlocked_sites,'''',''), ',')))::text  AS sitio,
fiber_node AS nodo_fibra,
NULL::numeric AS longitud_fibra_km,
1 AS saltos_radioenlace,
NULL::geometry AS geom_fibra
FROM {schema}.{table_output_2}
WHERE site NOT IN (
    SELECT
    TRIM(UNNEST(string_to_array(REPLACE(nodes_to_crop,'',''), ','))) AS site_cropped
    FROM {schema}.{table_output_crop}
)

) A
LEFT JOIN {schema_2}.{table_towers} I1
ON I1.tower_id::text = site
LEFT JOIN {schema_2}.{table_towers} I2
ON I2.tower_id::text = sitio
WHERE sitio IN (
        SELECT tower_id_1::text
        FROM {schema_2}.{table_line_of_sight}
        WHERE line_of_sight IS TRUE
        AND error_flag IS FALSE
        AND tower_id_2::text IN (
                SELECT site
                FROM {schema}.{table_output_2}
                WHERE site NOT IN (
                        SELECT
                        TRIM(UNNEST(string_to_array(REPLACE(nodes_to_crop,'',''), ','))) AS site_cropped
                        FROM {schema}.{table_output_crop}
                )
        )
        
        UNION
        
        SELECT tower_id_2::text
        FROM {schema_2}.{table_line_of_sight}
        WHERE line_of_sight IS TRUE
        AND error_flag IS FALSE
        AND tower_id_1::text IN (
                SELECT site
                FROM {schema}.{table_output_2}
                WHERE site NOT IN (
                        SELECT
                        TRIM(UNNEST(string_to_array(REPLACE(nodes_to_crop,'',''), ','))) AS site_cropped
                        FROM {schema}.{table_output_crop}
                )
        )
)

UNION

--Sitios desbloqueados con Radioenlace por carretera
SELECT 
id_despliegue,
priority,
segmento,
sitio,
nodo_fibra,
longitud_fibra_km,
saltos_radioenlace,
sitio || ', ROAD' AS camino_radioenlaces,
geom_fibra,
C.geom_line AS geom_radioenlaces
FROM (

SELECT DISTINCT ON (TRIM(UNNEST(string_to_array(REPLACE(unlocked_sites,'''',''), ',')))::text)
deployment_id AS id_despliegue,
2 AS priority,
'MW INDIRECT UNLOCKED'::text AS segmento,
TRIM(UNNEST(string_to_array(REPLACE(unlocked_sites,'''',''), ',')))::text  AS sitio,
fiber_node AS nodo_fibra,
NULL::numeric AS longitud_fibra_km,
1 AS saltos_radioenlace,
NULL::geometry AS geom_fibra
FROM {schema}.{table_output_2}
WHERE site NOT IN (
    SELECT
    TRIM(UNNEST(string_to_array(REPLACE(nodes_to_crop,'',''), ','))) AS site_cropped
    FROM {schema}.{table_output_crop}
)

) A
LEFT JOIN {schema}.{table_cluster_node_map_los} C
ON A.sitio = C.centroid
WHERE (sitio NOT  IN (
        SELECT centroid::text
        FROM {schema}.{table_output_2} P
        LEFT JOIN {schema}.{table_cluster_node_map_los} C
        ON P.node_2 = C.node_id::text
        WHERE site NOT IN (
            SELECT
            TRIM(UNNEST(string_to_array(REPLACE(nodes_to_crop,'',''), ','))) AS site_cropped
            FROM {schema}.{table_output_crop}
        )
        AND centroid IS NOT NULL
)
OR sitio IN (
        SELECT DISTINCT centroid
        FROM {schema}.{table_cluster_node_map_los}
        WHERE ST_Distance(geom_node::geography, geom_centroid::geography) > 1000
)
)
AND sitio NOT IN (
        SELECT tower_id_1::text
        FROM {schema_2}.{table_line_of_sight}
        WHERE line_of_sight IS TRUE
        AND error_flag IS FALSE
        AND tower_id_1::text IN (
                SELECT site
                FROM {schema}.{table_output_2}
                WHERE site NOT IN (
                        SELECT
                        TRIM(UNNEST(string_to_array(REPLACE(nodes_to_crop,'',''), ','))) AS site_cropped
                        FROM {schema}.{table_output_crop}
                )
        )
        
        UNION
        
        SELECT tower_id_2::text
        FROM {schema_2}.{table_line_of_sight}
        WHERE line_of_sight IS TRUE
        AND error_flag IS FALSE
        AND tower_id_2::text IN (
                SELECT site
                FROM {schema}.{table_output_2}
                WHERE site NOT IN (
                        SELECT
                        TRIM(UNNEST(string_to_array(REPLACE(nodes_to_crop,'',''), ','))) AS site_cropped
                        FROM {schema}.{table_output_crop}
                )
        )
)
ORDER BY sitio, priority
) P



UNION


--Despliegues de fibra normales
SELECT 
D.*,
F.geom_fibra,
CASE WHEN D.sitio IN (
        SELECT centroid
        FROM {schema}.{table_cluster_node_map_los} 
        WHERE NOT ST_DWithin(geom_node::geography, geom_centroid::geography, 1000)
     ) THEN C.geom_line::geometry
ELSE NULL::geometry END AS geom_radioenlaces
FROM (
SELECT DISTINCT ON (site)
deployment_id AS id_despliegue,
3 AS priority,
'FO DEPLOYMENT'::text AS segmento,
site AS sitio,
fiber_node AS nodo_fibra,
SUM(weight) AS longitud_fibra_km,
NULL::numeric AS saltos_radioenlace,
NULL::text AS camino_radioenlaces
FROM {schema}.{table_output_2}
WHERE site NOT IN (
        SELECT 
        centroid
        FROM {schema}.{table_quick_wins_fiber}
)
AND site NOT IN (
    SELECT
    TRIM(UNNEST(string_to_array(REPLACE(nodes_to_crop,'',''), ','))) AS site_cropped
    FROM {schema}.{table_output_crop}
)
GROUP BY deployment_id, site, fiber_node, population_density
)D
LEFT JOIN (
        SELECT 
        deployment_id,
        ST_Union(ST_MakeLine(N1.geom::geometry, N2.geom::geometry)) AS geom_fibra
        FROM {schema}.{table_output_2} P
        LEFT JOIN (
                SELECT  
                node_id,
                geom::geography
                FROM {schema}.{table_nodes_roads} 
                
                UNION
                
                SELECT
                tower_id + (SELECT MAX(node_id::numeric) FROM {schema}.{table_nodes_roads}) AS node_id,
                geom
                FROM {schema_2}.{table_towers}
                WHERE fiber IS TRUE
        )N1
        ON N1.node_id::text = P.node_1
        LEFT JOIN (
                SELECT  
                node_id,
                geom::geography
                FROM {schema}.{table_nodes_roads} 
                
                UNION
                
                SELECT
                tower_id + (SELECT MAX(node_id::numeric) FROM {schema}.{table_nodes_roads}) AS node_id,
                geom
                FROM {schema_2}.{table_towers}
                WHERE fiber IS TRUE
        ) N2
        ON N2.node_id::text = P.node_2
        WHERE site NOT IN (
                SELECT TRIM(UNNEST(string_to_array(REPLACE(nodes_to_crop,'',''), ',')))::text
                FROM {schema}.{table_output_crop}
        )
        GROUP BY deployment_id
        
) F
ON D.id_despliegue = F.deployment_id
LEFT JOIN {schema}.{table_cluster_node_map_los} C
ON C.centroid = D.sitio



UNION

--Despliegues de radioenlace

SELECT DISTINCT
9999 AS id_despliegue,
3 AS priority,
'MW DEPLOYMENT' AS segmento,
centroid AS sitio,
CASE WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_movistar THEN fiber_node_movistar
     WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_azteca THEN fiber_node_azteca
     WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_third_party THEN fiber_node_third_party END AS nodo_fibra,
NULL::numeric AS londitud_fibra_km,
CASE WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_movistar THEN hops_movistar
     WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_azteca THEN hops_azteca
     WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_third_party THEN hops_third_party END AS saltos_radioenlace,
CASE WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_movistar THEN path_movistar
     WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_azteca THEN path_azteca
     WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_third_party THEN path_third_party END AS camino_radioenlaces,
NULL::geometry AS geom_fibra,
CASE WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_movistar THEN geom_movistar
     WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_azteca THEN geom_azteca
     WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_third_party THEN geom_third_party END AS geom_radioenlaces
FROM {schema}.{table_output_microwave_2}
WHERE (hops_movistar IS NOT NULL OR hops_azteca IS NOT NULL OR hops_third_party IS NOT NULL)
AND centroid NOT IN (
        SELECT DISTINCT *
        FROM (
        
        SELECT DISTINCT
        site::text
        FROM {schema}.{table_output_2}
        WHERE site NOT IN (
            SELECT
            TRIM(UNNEST(string_to_array(REPLACE(nodes_to_crop,'',''), ','))) AS site_cropped
            FROM {schema}.{table_output_crop}
        )
        
        UNION
        
        SELECT DISTINCT
        TRIM(UNNEST(string_to_array(REPLACE(unlocked_sites,'''',''), ',')))::text  AS site
        FROM {schema}.{table_output_2}
                WHERE site NOT IN (
            SELECT
            TRIM(UNNEST(string_to_array(REPLACE(nodes_to_crop,'',''), ','))) AS site_cropped
            FROM {schema}.{table_output_crop}
        )

        )A

)

UNION
--Sitios que necesitarían salto intermedio
SELECT DISTINCT ON(sitio)
99999 AS id_despliegue,
3 AS priority,
'NEEDS INTERMEDIATE SITE' AS segmento,
centroid AS sitio,
CASE WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_movistar THEN fiber_node_movistar
     WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_azteca THEN fiber_node_azteca
     WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_third_party THEN fiber_node_third_party END AS nodo_fibra,
NULL::numeric AS longitud_fibra_km,
CASE WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_movistar THEN hops_movistar
     WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_azteca THEN hops_azteca
     WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_third_party THEN hops_third_party END AS saltos_radioenlace,
'INTERMEDIATE SITE' AS camino_radioenlaces,
NULL::geometry AS geom_fibra,
NULL::geometry AS geom_radioenlaces
FROM {schema}.{table_output_microwave_2} P
LEFT JOIN {schema_2}.{table_towers} I1
ON P.centroid = I1.tower_id::text
LEFT JOIN (
SELECT *
FROM {schema_2}.{table_towers}
WHERE fiber IS TRUE
) I2
ON ST_DWithin(I1.geom::geography, I2.geom::geography, 25000)
WHERE (hops_movistar IS NULL AND hops_azteca IS NULL AND hops_third_party IS NULL)
AND (I2.tower_id IS NOT NULL OR ST_DWithin(I1.geom::geography, (SELECT geom FROM {schema}.{table_geom_fiber}), 25000))
AND centroid NOT IN (
        SELECT DISTINCT *
        FROM (
        
        SELECT DISTINCT
        site::text
        FROM {schema}.{table_output_2}
        WHERE site NOT IN (
            SELECT
            TRIM(UNNEST(string_to_array(REPLACE(nodes_to_crop,'',''), ','))) AS site_cropped
            FROM {schema}.{table_output_crop}
        )
        
        UNION
        
        SELECT DISTINCT
        TRIM(UNNEST(string_to_array(REPLACE(unlocked_sites,'''',''), ',')))::text  AS site
        FROM {schema}.{table_output_2}
                WHERE site NOT IN (
            SELECT
            TRIM(UNNEST(string_to_array(REPLACE(nodes_to_crop,'',''), ','))) AS site_cropped
            FROM {schema}.{table_output_crop}
        )
        )A


)

UNION


--Sitios que se tienen que conectar con satélite

SELECT DISTINCT ON(sitio)
99999 AS id_despliegue,
4 AS priority,
'SATELLITE' AS segmento,
centroid AS sitio,
CASE WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_movistar THEN fiber_node_movistar
     WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_azteca THEN fiber_node_azteca
     WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_third_party THEN fiber_node_third_party END AS nodo_fibra,
NULL::numeric AS longitud_fibra_km,
CASE WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_movistar THEN hops_movistar
     WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_azteca THEN hops_azteca
     WHEN LEAST(hops_movistar, hops_azteca, hops_third_party) = hops_third_party THEN hops_third_party END AS saltos_radioenlace,
NULL::text AS camino_radioenlaces,
NULL::geometry AS geom_fibra,
NULL::geometry AS geom_radioenlaces
FROM {schema}.{table_output_microwave_2} P
LEFT JOIN {schema_2}.{table_towers} I1
ON P.centroid = I1.tower_id::text
LEFT JOIN (
SELECT *
FROM {schema_2}.{table_towers}
WHERE fiber IS TRUE
) I2
ON ST_DWithin(I1.geom::geography, I2.geom::geography, 25000)
WHERE (hops_movistar IS NULL AND hops_azteca IS NULL AND hops_third_party IS NULL)
AND (I2.tower_id IS NULL AND NOT ST_DWithin(I1.geom::geography, (SELECT geom FROM {schema}.{table_geom_fiber}), 25000))
AND centroid NOT IN (
        SELECT DISTINCT *
        FROM (
        
        SELECT DISTINCT
        site::text
        FROM {schema}.{table_output_2}
        WHERE site NOT IN (
            SELECT
            TRIM(UNNEST(string_to_array(REPLACE(nodes_to_crop,'',''), ','))) AS site_cropped
            FROM {schema}.{table_output_crop}
        )
        
        UNION
        
        SELECT DISTINCT
        TRIM(UNNEST(string_to_array(REPLACE(unlocked_sites,'''',''), ',')))::text  AS site
        FROM {schema}.{table_output_2}
        WHERE site NOT IN (
            SELECT
            TRIM(UNNEST(string_to_array(REPLACE(nodes_to_crop,'',''), ','))) AS site_cropped
            FROM {schema}.{table_output_crop}
        )
        )A


)

UNION

--sitios con 3G+
SELECT
999999 AS id_despliegue,
100 AS priority,
'SITE 3G+' AS segmento,
tower_id::text AS sitio,
NULL AS nodo_fibra,
NULL::numeric AS longitud_fibra_km,
NULL::numeric AS saltos_radioenlace,
NULL::text AS camino_radioenlaces,
NULL::geometry AS geom_fibra,
NULL::geometry AS geom_radioenlaces
FROM {schema_2}.{table_towers}
WHERE ipt_perimeter = 'IPT'
AND (tech_3g IS TRUE OR tech_4g IS TRUE)

UNION

--sitios con transporte
SELECT
999999 AS id_despliegue,
100 AS priority,
'SITE WITH EXISTING TRANSPORT' AS segmento,
tower_id::text AS sitio,
NULL AS nodo_fibra,
NULL::numeric AS longitud_fibra_km,
NULL::numeric AS saltos_radioenlace,
NULL::text AS camino_radioenlaces,
NULL::geometry AS geom_fibra,
NULL::geometry AS geom_radioenlaces
FROM {schema_2}.{table_towers}
WHERE ipt_perimeter = 'IPT'
AND (radio IS TRUE OR fiber IS TRUE)

ORDER BY priority, id_despliegue

) A
ORDER BY sitio, id_despliegue, segmento, sitio ASC
) B
ORDER BY id_despliegue, segmento
"""

query_formatted = query.format(schema = schema,
                               schema_2 = schema_2,
                               table_output_microwave = table_output_microwave,
                               table_towers = table_towers,
                               table_output_microwave_2 = table_output_microwave_2,
                               table_output_2 = table_output_2,
                               table_output_crop = table_output_crop,
                               table_line_of_sight = table_line_of_sight,
                               table_nodes_roads = table_nodes_roads,
                               table_cluster_node_map_los = table_cluster_node_map_los,
                               table_geom_fiber = table_geom_fiber,
                               table_output_basic = table_output_basic,
                               table_quick_wins_radio = table_quick_wins_radio,
                               table_quick_wins_fiber = table_quick_wins_fiber,
                               table_initial_quick_wins = table_initial_quick_wins,
                               table_nodes = table_nodes).replace("[","").replace("]","")
connection = engine.connect()
#print(query_formatted)

engine.execute(query_formatted)


connection.close()


In [ ]:
#Create final table of the whole process


query = """
DROP TABLE IF EXISTS {schema}.{table_output_final};
CREATE TABLE {schema}.{table_output_final} AS

SELECT DISTINCT ON (id_despliegue, O.sitio, segmento)
id_despliegue,
CASE WHEN segmento = 'FO DEPLOYMENT' AND site IN (SELECT centroid FROM {schema}.{table_cluster_node_map_los} WHERE NOT ST_DWithin(geom_node::geography, geom_centroid::geography, 1000)) THEN 'FIBRA + RADIO' 
     WHEN segmento IN ('FO QUICK WINS', 'FO DEPLOYMENT', 'NEAR FO POP QUICK WINS', 'FO INDIRECT UNLOCKED') THEN 'FIBRA'
     WHEN segmento IN ('MW DEPLOYMENT', 'MW INDIRECT UNLOCKED', 'MW QUICK WINS', 'NEEDS INTERMEDIATE SITE', 'NEAR MW POP QUICK WINS') THEN 'RADIO'
     WHEN segmento IN ('SATELLITE') THEN 'SATELITE'
     ELSE 'SITIO OPERATIVO' END AS medio_transporte,     
segmento,
sitio AS id_sitio,
I1.tower_name AS sitio,
I1.internal_id AS codigo_estacion_sitio,
I1.type AS tipo_estacion,
C.cluster_weight AS poblacion_directa,
P.unlocked_population AS poblacion_indirecta,
P.population_density AS densidad_poblacion,
I1.latitude AS latitud_sitio,
I1.longitude AS longitud_sitio,
O.nodo_fibra AS id_nodo_fibra,
I2.tower_name AS nodo_fibra,
I2.internal_id AS codigo_estacion_fibra,
I2.latitude AS latitud_fibra,
I2.longitude AS longitud_fibra,
CASE WHEN I2.source LIKE 'AZTECA' THEN I2.owner
     WHEN I2.source IN ('GILAT', 'TORRES ANDINAS', 'EHAS') THEN I2.owner
     WHEN I2.source IN ('REGIONAL', 'YOFC', 'ADVISIA') THEN 'REGIONAL'
     ELSE 'MOVISTAR' END AS proveedor_fibra,
O.longitud_fibra_km,
O.saltos_radioenlace,
O.camino_radioenlaces,
I1.geom AS geom_sitio,
I2.geom AS geom_nodo_fibra,
O.geom_radioenlaces,
O.geom_fibra
FROM {schema}.{table_output_basic} O
LEFT JOIN {schema_2}.{table_towers} I1
ON I1.tower_id::text = O.sitio
LEFT JOIN {schema_2}.{table_towers} I2
ON I2.tower_id::text = O.nodo_fibra
LEFT JOIN {schema_2}.{table_clusters} C
ON C.centroid = O.sitio
LEFT JOIN {schema}.{table_output_2} P
ON P.site = O.sitio

"""

query_formatted = query.format(schema = schema,
                               schema_2 = schema_2,
                               table_output_microwave = table_output_microwave,
                               table_towers = table_towers,
                               table_output_microwave_2 = table_output_microwave_2,
                               table_output_2 = table_output_2,
                               table_clusters = table_clusters,
                               table_output_final = table_output_final,
                               table_output_crop = table_output_crop,
                               table_line_of_sight = table_line_of_sight,
                               table_nodes_roads = table_nodes_roads,
                               table_cluster_node_map_los = table_cluster_node_map_los,
                               table_geom_fiber = table_geom_fiber,
                               table_output_basic = table_output_basic,
                               table_quick_wins_radio = table_quick_wins_radio,
                               table_quick_wins_fiber = table_quick_wins_fiber,
                               table_initial_quick_wins = table_initial_quick_wins,
                               table_nodes = table_nodes).replace("[","").replace("]","")

connection = engine.connect()
#print(query_formatted)

engine.execute(query_formatted)


connection.close()

In [ ]:
#We refine the output by updating the FO PoPs associated to the sites in 'NEEDS INTERMEDIATE SITE' or 'MW DEPLOYMENT'
#We update the fiber node for MW DEPLOYMENT sites
#For those coming from a site in a FO deployment, the fiber node is that site
#It should be assigned to the fiber node used to connect the FO deployment

#First we update easily the MW DEPLOYMENT sites
#Then we update the 'NEEDS INTERMEDIATE SITE' sites
#Part 1: First, for those who have a deployed site 25km away or less, we assign that site's FO PoP
#Part 2: Then, for those whose nearest site does not have FO, we assign any FO PoP 25km away or less


query = """
DROP TABLE IF EXISTS {schema}.{table_output_refined};

CREATE TABLE {schema}.{table_output_refined} AS

--All sites that are not MW DEPLOYMENT or NEEDS INTERMEDIATE SITE remain as they are
SELECT *
FROM {schema}.{table_output_final}
WHERE segmento <> 'NEEDS INTERMEDIATE SITE'
AND id_sitio NOT IN (
        SELECT
        P2.id_sitio
        FROM {schema}.{table_output_final} P1
        LEFT JOIN {schema}.{table_output_final} P2
        ON (P1.id_sitio = P2.id_nodo_fibra)
        WHERE P1.id_sitio <> P2.id_sitio
        AND P1.id_nodo_fibra IS NOT NULL
        AND P2.segmento = 'MW DEPLOYMENT'
)

UNION

--We update the fiber node for MW DEPLOYMENT sites
--For those coming from a site in a FO deployment, the fiber node is that site
--It should be assigned to the fiber node used to connect the FO deployment
SELECT
P2.id_despliegue,
P2.medio_transporte,
P2.segmento,
P2.id_sitio,
P2.sitio,
P2.codigo_estacion_sitio,
P2.tipo_estacion,
P2.poblacion_directa,
P2.poblacion_indirecta,
P2.densidad_poblacion,
P2.latitud_sitio,
P2.longitud_sitio,
P1.id_nodo_fibra,
P1.nodo_fibra,
P1.codigo_estacion_fibra,
P1.latitud_fibra,
P1.longitud_fibra,
P1.proveedor_fibra,
P2.longitud_fibra_km,
P2.saltos_radioenlace, 
P2.camino_radioenlaces,
P2.geom_sitio,
P1.geom_nodo_fibra,
P2.geom_radioenlaces,
P2.geom_fibra
FROM {schema}.{table_output_final} P1
LEFT JOIN {schema}.{table_output_final} P2
ON (P1.id_sitio = P2.id_nodo_fibra)
WHERE P1.id_sitio <> P2.id_sitio
AND P1.id_nodo_fibra IS NOT NULL
AND P2.segmento = 'MW DEPLOYMENT'



UNION

--We update the 'NEEDS INTERMEDIATE SITE' sites
--Part 1: First, for those who have a deployed site 25km away or less, we assign that site's FO PoP
--Part 2: Then, for those whose nearest site does not have FO, we assign any FO PoP 25km away or less

--PART 1
SELECT 
id_despliegue,
medio_transporte,
segmento,
id_sitio,
sitio,
codigo_estacion_sitio,
tipo_estacion,
poblacion_directa,
poblacion_indirecta,
densidad_poblacion,
latitud_sitio,
longitud_sitio,
id_nodo_fibra,
nodo_fibra,
codigo_estacion_fibra,
latitud_fibra,
longitud_fibra,
proveedor_fibra,
longitud_fibra_km,
saltos_radioenlace,
camino_radioenlaces,
geom_sitio,
geom_nodo_fibra,
geom_radioenlaces,
geom_fibra
FROM (

SELECT DISTINCT ON (P1.id_sitio)
P1.id_despliegue,
P1.medio_transporte,
P1.segmento,
P1.id_sitio,
P1.sitio,
P1.codigo_estacion_sitio,
P1.tipo_estacion,
P1.poblacion_directa,
P1.poblacion_indirecta,
P1.densidad_poblacion,
P1.latitud_sitio,
P1.longitud_sitio,
P2.id_nodo_fibra,
P2.nodo_fibra,
P2.codigo_estacion_fibra,
P2.latitud_fibra,
P2.longitud_fibra,
P2.proveedor_fibra,
P1.longitud_fibra_km,
P1.saltos_radioenlace,
P1.camino_radioenlaces,
P1.geom_sitio,
P2.geom_nodo_fibra,
P1.geom_radioenlaces,
P1.geom_fibra,
ST_Distance(P1.geom_sitio::geography, P2.geom_fibra::geography) AS distance
FROM {schema}.{table_output_final} P1
LEFT JOIN {schema}.{table_output_final} P2 
ON ST_DWithin(P1.geom_sitio::geography,P2.geom_fibra::geography, 25000)
WHERE P1.segmento = 'NEEDS INTERMEDIATE SITE'
AND P1.id_sitio NOT IN (
        SELECT id_sitio
        FROM (
        SELECT DISTINCT ON (P1.id_sitio)
        P1.id_sitio,
        ST_Distance(P1.geom_sitio::geography, P2.geom_fibra::geography) AS distance
        FROM {schema}.{table_output_final} P1
        LEFT JOIN {schema}.{table_output_final} P2 
        ON ST_DWithin(P1.geom_sitio::geography,P2.geom_fibra::geography, 25000)
        WHERE P1.segmento = 'NEEDS INTERMEDIATE SITE'
        AND P2.id_nodo_fibra IS NULL
        ORDER BY P1.id_sitio, distance
        )A
)
ORDER BY P1.id_sitio, distance

)B

UNION

--PART 2
SELECT 
id_despliegue,
medio_transporte,
segmento,
id_sitio,
sitio,
codigo_estacion_sitio,
tipo_estacion,
poblacion_directa,
poblacion_indirecta,
densidad_poblacion,
latitud_sitio,
longitud_sitio,
id_nodo_fibra,
nodo_fibra,
codigo_estacion_fibra,
latitud_fibra,
longitud_fibra,
proveedor_fibra,
longitud_fibra_km,
saltos_radioenlace,
camino_radioenlaces,
geom_sitio,
geom_nodo_fibra,
geom_radioenlaces,
geom_fibra
FROM (

SELECT DISTINCT ON (P1.id_sitio)
P1.id_despliegue,
P1.medio_transporte,
P1.segmento,
P1.id_sitio,
P1.sitio,
P1.codigo_estacion_sitio,
P1.tipo_estacion,
P1.poblacion_directa,
P1.poblacion_indirecta,
P1.densidad_poblacion,
P1.latitud_sitio,
P1.longitud_sitio,
P2.id_nodo_fibra,
P2.nodo_fibra,
P2.codigo_estacion_fibra,
P2.latitud_fibra,
P2.longitud_fibra,
P2.proveedor_fibra,
P1.longitud_fibra_km,
P1.saltos_radioenlace,
P1.camino_radioenlaces,
P1.geom_sitio,
P2.geom_nodo_fibra,
P1.geom_radioenlaces,
P1.geom_fibra,
ST_Distance(P1.geom_sitio::geography, P2.geom_fibra::geography) AS distance
FROM {schema}.{table_output_final} P1
LEFT JOIN (

SELECT 
*
FROM {schema}.{table_output_final} O 

UNION

SELECT
0,
'NONE',
'NONE',
'NONE',
'NONE',
'NONE',
'NONE',
0,
0,
0,
0,
0,
tower_id::text,
tower_name,
internal_id,
latitude,
longitude,
CASE WHEN I2.source LIKE 'AZTECA' THEN I2.owner
     WHEN I2.source IN ('GILAT', 'TORRES ANDINAS', 'EHAS') THEN I2.owner
     WHEN I2.source IN ('REGIONAL', 'YOFC', 'ADVISIA') THEN 'REGIONAL'
     ELSE 'MOVISTAR' END AS proveedor_fibra,
0,
0,
'NONE',
geom::geometry,
geom::geometry,
geom::geometry,
geom::geometry
FROM {schema_2}.{table_towers} I2
WHERE fiber IS TRUE


) P2 
ON ST_DWithin(P1.geom_sitio::geography,P2.geom_fibra::geography, 25000)
WHERE P1.segmento = 'NEEDS INTERMEDIATE SITE'
AND P1.id_sitio IN (
        SELECT id_sitio
        FROM (
        SELECT DISTINCT ON (P1.id_sitio)
        P1.id_sitio,
        ST_Distance(P1.geom_sitio::geography, P2.geom_fibra::geography) AS distance
        FROM {schema}.{table_output_final} P1
        LEFT JOIN {schema}.{table_output_final} P2 
        ON ST_DWithin(P1.geom_sitio::geography,P2.geom_fibra::geography, 25000)
        WHERE P1.segmento = 'NEEDS INTERMEDIATE SITE'
        AND P2.id_nodo_fibra IS NULL
        ORDER BY P1.id_sitio, distance
        )A
)
ORDER BY P1.id_sitio, distance

)C

"""

query_formatted = query.format(schema = schema,
                               schema_2 = schema_2,
                               table_output_microwave = table_output_microwave,
                               table_output_refined = table_output_refined,
                               table_towers = table_towers,
                               table_output_microwave_2 = table_output_microwave_2,
                               table_output_final = table_output_final,
                               table_output_crop = table_output_crop,
                               table_line_of_sight = table_line_of_sight,
                               table_nodes_roads = table_nodes_roads,
                               table_cluster_node_map_los = table_cluster_node_map_los,
                               table_output_2 = table_output_2,
                               table_geom_fiber = table_geom_fiber,
                               table_output_basic = table_output_basic,
                               table_quick_wins_radio = table_quick_wins_radio,
                               table_quick_wins_fiber = table_quick_wins_fiber,
                               table_initial_quick_wins = table_initial_quick_wins,
                               table_nodes = table_nodes).replace("[","").replace("]","")
connection = engine.connect()
#print(query_formatted)

engine.execute(query_formatted)


connection.close()

In [ ]:
#We reshape the output into clusters

query = """

DROP TABLE IF EXISTS {schema}.{table_output_clusters};
CREATE TABLE {schema}.{table_output_clusters} AS
SELECT 
id_nodo_fibra,
numero_sitios,
numero_macros,
numero_femtos,
poblacion_directa_total,
poblacion_macros,
poblacion_femtos,
longitud_fibra_km,
numero_sitios_conectados_con_fibra,
numero_sitios_conectados_con_radio,
numero_radioenlaces + ROUND(1.5*numero_sitios_con_necesidad_de_torre_intermedia, 0) AS numero_radioenlaces,
numero_sitios_con_necesidad_de_torre_intermedia,
ids_sitios,
sitios,
codigos_estacion_sitios,
geom_sitio,
geom_nodo_fibra,
geom_fibra AS geom_fibra,
geom_radioenlaces AS geom_radioenlaces,
CASE WHEN geom_fibra IS NULL THEN geom_radioenlaces
     WHEN geom_radioenlaces IS NULL THEN geom_fibra
     ELSE ST_Union(geom_radioenlaces, geom_fibra) END AS geom_cluster
FROM (

SELECT 
id_nodo_fibra,
COUNT(1) AS numero_sitios,
COUNT(1) FILTER (WHERE tipo_estacion = 'MACRO') AS numero_macros,
COUNT(1) FILTER (WHERE tipo_estacion = 'FEMTO') AS numero_femtos,
SUM(poblacion_directa) AS poblacion_directa_total,
SUM(poblacion_directa) FILTER (WHERE tipo_estacion = 'MACRO') AS poblacion_macros,
SUM(poblacion_directa) FILTER (WHERE tipo_estacion = 'FEMTO') AS poblacion_femtos,
SUM(longitud_fibra_km) AS longitud_fibra_km,
COUNT(1) FILTER (WHERE medio_transporte LIKE '%%FIBRA%%') AS numero_sitios_conectados_con_fibra,
COUNT(1) FILTER (WHERE medio_transporte LIKE '%%RADIO%%' OR segmento LIKE '%%NEEDS INTERMEDIATE SITE%%') AS numero_sitios_conectados_con_radio,
CASE WHEN SUM(saltos_radioenlace) IS NULL THEN 0
     ELSE SUM(saltos_radioenlace) END AS numero_radioenlaces,
COUNT(1) FILTER(WHERE segmento LIKE '%%NEEDS INTERMEDIATE SITE%%') AS numero_sitios_con_necesidad_de_torre_intermedia,
string_agg(id_sitio, ', ') AS ids_sitios,
string_agg(sitio, ', ') AS sitios,
string_agg(codigo_estacion_sitio, ', ') AS codigos_estacion_sitios,
ST_Union(geom_sitio::geometry) AS geom_sitio,
ST_Union(geom_nodo_fibra::geometry) AS geom_nodo_fibra,
ST_Union(geom_fibra) AS geom_fibra,
ST_Union(geom_radioenlaces) AS geom_radioenlaces
FROM {schema}.{table_output_refined}
WHERE id_nodo_fibra IS NOT NULL
GROUP BY id_nodo_fibra
ORDER BY numero_sitios DESC

)A

"""

query_formatted = query.format(schema = schema,
                               schema_2 = schema_2,
                               table_output_microwave = table_output_microwave,
                               table_output_refined = table_output_refined,
                               table_output_clusters = table_output_clusters,
                               table_towers = table_towers,
                               table_output_microwave_2 = table_output_microwave_2,
                               table_output_final = table_output_final,
                               table_output_crop = table_output_crop,
                               table_line_of_sight = table_line_of_sight,
                               table_nodes_roads = table_nodes_roads,
                               table_cluster_node_map_los = table_cluster_node_map_los,
                               table_output_2 = table_output_2,
                               table_geom_fiber = table_geom_fiber,
                               table_output_basic = table_output_basic,
                               table_quick_wins_radio = table_quick_wins_radio,
                               table_quick_wins_fiber = table_quick_wins_fiber,
                               table_initial_quick_wins = table_initial_quick_wins,
                               table_nodes = table_nodes).replace("[","").replace("]","")
connection = engine.connect()
#print(query_formatted)

engine.execute(query_formatted)


connection.close()